# AI Avengers: GridSense (Kenya)

# Chapter 1: Business Understanding 

## Background & context
Kenya is a continental leader in clean electricity, with high shares of geothermal, hydro, wind and solar in its generation mix. EPRA’s latest statistics show renewables dominate annual generation (approx. 83% in 2023), with geothermal the single largest source; recent reports also note record peak demand rising from ~2,177MW (2024) to ~2,316MW (2025) evidence of a growing, weather-sensitive grid that must balance maintenance, load growth, and climate variability. Meanwhile, access to electricity in Kenya has climbed to ~76% (2023), but reliability for end-users remains uneven and costly for SMEs and social services. Across Sub-Saharan Africa, progress on universal access has stalled and financing needs far exceed current commitments, keeping reliability and affordability under pressure. 

Kenya also experiences episodic widespread outages (e.g., Dec 18 2024, multi-hour blackout affecting most of the country), underscoring the value of weather-aware preparedness and better translation of utility maintenance plans into actionable risk for businesses and counties. 

## Problem statement
Today, customers receive planned-maintenance notices that outline where/when power will be interrupted, but they rarely get hour-by-hour risk probabilities, nor clear “what to do now” recommendations tailored to their operations. Weather, vegetation conditions and historical patterns can extend or exacerbate outage windows (e.g., storms delaying restoration) or increase grid stress even when no notice is present. Businesses, clinics and telecom sites need a localized, short-horizon risk signal and simple, low-cost actions to reduce losses.

## Overall objective (MVP)
Develop a predictive model that forecasts county-level grid-stress / outage risk 6–24 hours ahead using weather conditions, vegetation proxies, historical outage patterns, and available consumption signal , and a small recommender that converts “high-risk windows” into top 2–3 actions per user persona (e.g., SME, cold-chain), with brief, auditable reasons.

## SMART objectives 
1. Model performance:Train a county-hour classifier achieving PR-AUC ≥ 0.4  and Recall ≥ 0.60 at a 5% alert budge  on a held-out recent period (time-based split).  
2. Calibration & validity: Deliver Brier score ≤ 0.2  and include 2–3 face-validity backtest  around notable national outage days to show external relevance (plots only; no training leakage). 
3. Productization: Deploy a minimal Streamlit app that displays next-6/12/24h risk by county and returns Top-3 action per persona with reason codes (e.g., “planned window + rain_24h↑ + wind_max_24h↑”).  
4. Data quality & lineage: Produce a reproducible pipeline with versioned datasets (labels_admin_hourly, features_weather_hourly, training_matrix ) ensuring no duplicate (county, hour  rows and Africa/Nairobi timestamps end-to-end).

## Metrics of success
- Discrimination  PR-AUC on held-out period; Recall@  where *k* equals 5% alert rate (operational budgeted alarms).  
- Calibration Brier score and reliability curve.  
- Coverage & latency  % of counties with valid hourly predictions; daily pipeline runtime within target.  
- Actionability % of high-risk windows for which the app proposes at least one low-cost action; qualitative feedback from 1–2 case-study sites (Athi River, Garissa).  
- Explainability Global feature importances + per-prediction reason snippets aligned to engineering features (e.g., CAPE/precipitation rollups, wind gust maxima, vegetation proxy).

## Targeted stakeholders & users
- Primary users   
  - SMEs / manufacturers / cold-chain operator (reduce spoilage, reschedule tasks, optimize genset starts).   
  - Telecom tower/site operators (battery/genset scheduling, maintenance).  
- Ecosystem stakeholders: 
  - County energy & disaster desks (preparedness dashboards, incident coordination).  
  - Insurers/financiers (operational risk scoring, resilience incentives).  
  - Utility & regulators (future collaboration) for co-designing maintenance communications and resilience metrics. 

## How our solution helps society (near-term to long-run)
- Near-term (MVP): Convert public maintenance notices + weather into hourly, county-level risk and practical actions that immediately reduce avoidable losses for SMEs, clinics and telecoms.  
- Medium-term: Add vegetation/land cover and finer network layers (substations/lines) to improve spatial precision; incorporate consumption-based stress indicators where available.  
- Long-run: Evolve into a national outage-risk & preparedness service that integrates official plans, nowcasts, grid topology and (when accessible) unplanned fault streams, aligning with Kenya’s clean-energy leadership and Africa’s universal-access agenda

# Chapter 2: Data Understanding
The Data Understanding phase establishes what data we have, how it’s structured, and whether it’s fit for purpose before we engineer features or train models. We profile each source, validate keys and timestamps, and surface quality issues early so downstream steps are reproducible and defensible.

What this section covers
- Scope & granularity: Define the unit of analysis (county × hour, Africa/Nairobi) and the modeling horizon (next 6–24 hours).
- Sources & lineage: Describe each dataset (KPLC notices, weather/engineered features, vegetation/land cover, consumption samples, grid layers), where it comes from, how it’s refreshed, and how raw, cleaned and merged artifacts are versioned.
- Schemas & keys: List columns, data types, primary/foreign keys, and the join strategy (e.g., (county, timestamp_local)).
- Time & timezone checks: Verify completeness of hourly timestamps, gaps/overlaps, and consistent Africa/Nairobi localization.
- Label construction: Explain how planned-outage windows are expanded into a labeled county-hour target (planned_outage {0,1}).
- Descriptive profiling: Summary stats (counts, missingness, class balance), distributions, and basic correlations to understand signal vs noise.
- Data quality risks: Duplicates, inconsistent county names, outliers, sparse positives, and how we’ll address them (standardization, imputation rules, thresholds).
- Assumptions & limitations: What the data can/can’t tell us (e.g., limited unplanned-fault labels), and how we mitigate (face-validity checks, calibration).
- Ethics & governance: Public data usage, privacy considerations, aggregation level, and auditability.
- Readiness criteria: A checklist confirming the data is sufficient to progress to Data Preparation/Feature Engineering (cleaned tables, validated joins, documented caveats).

In [1]:
#import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from warnings import filterwarnings
filterwarnings('ignore')

import geopandas as gpd
from shapely.geometry import Point

import json
from pathlib import Path

#import sklearn libraries
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, \
    RandomizedSearchCV, StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, \
    roc_auc_score, roc_curve,precision_score, recall_score, f1_score
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression

from imblearn.over_sampling import SMOTE, SMOTEN

#import stats libraries
import statsmodels.api as sm

#import scipy libraries
from scipy import stats
from scipy.stats import norm, skew

#import tensorflow libraries
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, LSTM
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint


## 2.1 Outages

### Planned Outage Notices — Column Dictionary

| Column            | Description |
|-------------------|-------------|
| **region**        | Kenya Power operational region grouping multiple counties (e.g., Nairobi Region, North Rift). |
| **county**        | Kenyan administrative county where the interruption applies. |
| **area**          | Sub-county/locality label used in the notice (e.g., “Part of Karen”, “Makunga”). Helpful as a human-friendly cluster within a county. |
| **date**          | Publication or effective date of the notice (string in source). |
| **Date**          | Parsed/alternate date column (often same as `date` but from a different extraction step). One of `date`/`Date` should be standardized and the other dropped during cleaning. |
| **start_time**    | **Start of the planned outage window** in local time (Africa/Nairobi), as written in the notice. |
| **end_time**      | **End of the planned outage window** in local time (Africa/Nairobi), as written in the notice. |
| **affected_locations** | Free-text list of estates, roads, landmarks, markets, schools, etc., that fall within the outage area (e.g., “Lower Kabete Rd, Shanzu Rd, … & adjacent customers”). Use this for finer spatial mapping. |
| **source_file**   | The PDF/notice filename or URL from which the row was extracted (audit trail). May be missing for some rows if not captured during scraping. |
| **Unnamed: 9**    | Empty/auxiliary column created during parsing; should be **dropped** in cleaning. |
| **Unnamed: 10**   | Empty/auxiliary column created during parsing; should be **dropped** in cleaning. |

In [2]:
#import datasets
# preview the first few rows of the dataset
outage_df = pd.read_csv('../Data/KPLC Power Maintenance Schedule/outages.csv')
outage_df.head()

,region,county,area,date,Date,start_time,end_time,affected_locations,source_file,Unnamed: 9,Unnamed: 10
0,NORTH RIFT,UASIN GISHU,SINENDET CENTRE,29/06/2024,29.06.2024,10:00,5:00 pm,"Sinendet Centre, Sakam Quarry, Lower Kipkaren ...",NaN,NaN,NaN
1,NORTH RIFT,TRANS NZOIA,"KITALE TOWN, MARAMBACHI",29/06/2024,29.06.2024,09:00,12:00 pm,"Wamuini Centre, St. Joseph's Boys High Sch, St...",NaN,NaN,NaN
2,WESTERN,BUSIA,"TANGA, KONA, NAMBALE",29/06/2024,29.06.2024,09:00,5:00 pm,"Nasira, Kajoro, Kololo Farm, Nambale DC, Jua K...",NaN,NaN,NaN
3,NORTH EASTERN,KIAMBU,"KIKUYU TOWN, SIGONA",29/06/2024,29.06.2024,12:00,16:00,"Kikuyu Town, Kidfarmaco, Gitaru, Regen, Magana...",NaN,NaN,NaN
4,NAIROBI,NAIROBI,PART OF EASTLEIGH,30/06/2024,30.06.2024,09:00,5:00 pm,"Mohammed Yusuf Haji Ave, Ismail Hassan, Moyale...",NaN,NaN,NaN


In [3]:
# check the tail to confirm uniformity
outage_df.tail()

,region,county,area,date,Date,start_time,end_time,affected_locations,source_file,Unnamed: 9,Unnamed: 10
2298,NORTH EASTERN,KIAMBU,"KIGWARU, RUAKA, MUCHATHA",26/10/2025,NaN,09:00,16:00,"Little Sisters, Guango, Clifftop, Kigwaru-Inn,...",NaN,NaN,NaN
2299,NORTH EASTERN,KIAMBU,JUJA INDUSTRIAL PARK,26/10/2025,NaN,09:00,16:00,"Juja International Abattoir Limited, Juja Deve...",NaN,NaN,NaN
2300,NORTH EASTERN,KIAMBU,KIHINGO,29/10/2025,NaN,09:00,17:00,"Kihingo ACK, Kihingo 1st, 2nd, & 3rd Aves & ad...",NaN,NaN,NaN
2301,NORTH EASTERN,KIAMBU,"NAZARETH, KIMORORI",30/10/2025,NaN,09:00,17:00,"Gatatha Farmers, Kimorori, Kawaida, Black Peta...",NaN,NaN,NaN
2302,NORTH EASTERN,KIAMBU,"FRESHA DAIRIES, PALMHOUSE DAIRIES",31/10/2025,NaN,09:00,17:00,"Thakwa Village, County Pride Hotel, Githunguri...",NaN,NaN,NaN


**Observations**
- The dataset is uniform at the top and bottom
- We will drop 
    - all columns after affected columns since they have no value
    - column 'Date' since it has missing values and a replica

In [4]:
# checking the info of the dataset
outage_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2303 entries, 0 to 2302
Data columns (total 11 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   region              2303 non-null   object
 1   county              2303 non-null   object
 2   area                2303 non-null   object
 3   date                2303 non-null   object
 4   Date                1753 non-null   object
 5   start_time          2303 non-null   object
 6   end_time            2303 non-null   object
 7   affected_locations  2303 non-null   object
 8   source_file         850 non-null    object
 9   Unnamed: 9          2 non-null      object
 10  Unnamed: 10         1 non-null      object
dtypes: object(11)
memory usage: 198.0+ KB


**Observations**
- All columns are categorical
- We will change date, start_time and end_time to datetime datatypes
- We have 11 columns

In [5]:
# checking column names
outage_df.columns

Index(['region', 'county', 'area', 'date', 'Date', 'start_time', 'end_time',
       'affected_locations', 'source_file', 'Unnamed: 9', 'Unnamed: 10'],
      dtype='object')

**Observations:**
- All column names are in lowercase and named correctly

In [6]:
#data summary statistics
outage_df.describe(include='all').T

,count,unique,top,freq
region,2303,20,NAIROBI,371
county,2303,94,NAIROBI,180
area,2303,1820,PART OF PARKLANDS,14
date,2303,357,03/07/2025,27
Date,1753,308,03.07.2025,27
start_time,2303,15,09:00,1744
end_time,2303,23,5:00 pm,1801
affected_locations,2303,2094,"Kipkenyo, Kosyin, Leseru, Seiyot & adjacent cu...",5
source_file,850,68,01JMHWRB64HYFS4DXNT8G2AXPS,59
Unnamed: 9,2,2,"Maviani, Gategi, Kaseve, Kathiani, Bonzuki, Ka...",1


**Observations:**
- Nairobi Region and Nairobi county has the highest occurrences of outages
- Parts of Parklands is the most affected area in terms of outages
- On 3/07/2025, we had the most outages
- We have most outages happening between 9AM - 5PM

## 2.2 Kenya Weather 

| Column              | Meaning            | Short Explanation                                |
|---------------------|--------------------|--------------------------------------------------|
| county              | County name        | Kenyan administrative region                     |
| YEAR                | Year               | Year of observation                              |
| MO                  | Month              | Month number (1–12)                              |
| DY                  | Day                | Day of the month                                 |
| date                | Date               | Combined full date                               |
| PRECTOTCORR         | Precipitation      | Daily rainfall (mm/day)                          |
| ALLSKY_SFC_SW_DWN   | Solar irradiance   | Solar energy at surface (kWh/m²/day)             |
| ALLSKY_KT           | Clearness index    | Sky clarity (0–1 scale)                          |
| latitude            | Latitude           | County’s north–south position                    |
| longitude           | Longitude          | County’s east–west position                      |

In [7]:
# load weather dataset
weather_df = pd.read_csv('../Data/Weather/kenya_weather_all_counties_2024_2025.csv')
weather_df.head()

,county,YEAR,MO,DY,date,PRECTOTCORR,ALLSKY_SFC_SW_DWN,ALLSKY_KT,latitude,longitude
0,Turkana,2024,1,1,1/01/2024,2.61,5.2291,0.55,3.412964,35.434957
1,Turkana,2024,1,2,2/01/2024,0.02,5.7240,0.60,3.412964,35.434957
2,Turkana,2024,1,3,3/01/2024,0.02,5.7619,0.61,3.412964,35.434957
3,Turkana,2024,1,4,4/01/2024,0.00,6.3756,0.67,3.412964,35.434957
4,Turkana,2024,1,5,5/01/2024,0.00,5.9974,0.63,3.412964,35.434957


In [8]:
#chech tail for uniformity
weather_df.tail()

,county,YEAR,MO,DY,date,PRECTOTCORR,ALLSKY_SFC_SW_DWN,ALLSKY_KT,latitude,longitude
31250,Mombasa,2025,10,22,22/10/2025,0.19,-999.0,-999.0,-4.017651,39.65462
31251,Mombasa,2025,10,23,23/10/2025,0.67,-999.0,-999.0,-4.017651,39.65462
31252,Mombasa,2025,10,24,24/10/2025,-999.00,-999.0,-999.0,-4.017651,39.65462
31253,Mombasa,2025,10,25,25/10/2025,-999.00,-999.0,-999.0,-4.017651,39.65462
31254,Mombasa,2025,10,26,26/10/2025,-999.00,-999.0,-999.0,-4.017651,39.65462


**Observations:**
- All columns are uniform 
- We will keep all columns
- Since we will merge by date, they have to be in the same format

In [9]:
# checking the info of the dataset
weather_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31255 entries, 0 to 31254
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   county             31255 non-null  object 
 1   YEAR               31255 non-null  int64  
 2   MO                 31255 non-null  int64  
 3   DY                 31255 non-null  int64  
 4   date               31255 non-null  object 
 5   PRECTOTCORR        31255 non-null  float64
 6   ALLSKY_SFC_SW_DWN  31255 non-null  float64
 7   ALLSKY_KT          31255 non-null  float64
 8   latitude           31255 non-null  float64
 9   longitude          31255 non-null  float64
dtypes: float64(5), int64(3), object(2)
memory usage: 2.4+ MB


**Observations:**
- We have no missing values
- change date datatype

In [10]:
#data summary statistics
weather_df.describe().T

,count,mean,std,min,25%,50%,75%,max
YEAR,31255.0,2024.449624,0.497464,2024.000000,2024.000000,2024.000000,2025.000000,2025.000000
MO,31255.0,6.036090,3.232671,1.000000,3.000000,6.000000,9.000000,12.000000
DY,31255.0,15.637594,8.760239,1.000000,8.000000,16.000000,23.000000,31.000000
PRECTOTCORR,31255.0,-0.639584,67.665993,-999.000000,0.140000,0.960000,3.960000,267.050000
ALLSKY_SFC_SW_DWN,31255.0,-3.359157,95.010509,-999.000000,5.011200,5.835400,6.505100,8.176600
ALLSKY_KT,31255.0,-131.699564,338.711512,-999.000000,0.460000,0.570000,0.640000,0.780000
latitude,31255.0,-0.336641,1.621684,-4.139058,-1.253919,-0.343076,0.532028,3.438400
longitude,31255.0,36.785633,1.883245,34.193840,35.112300,36.819685,37.871708,40.766286


**Observations:**
- highest daily rainfall is 267.05mm in a day
- The highest solar energy at surface is 5 in a day
- The highest sky clarity is 0.46 
- (-999) is a strange entry that could be for null values

## 2.3 Weather Hourly Data Understanding

### Overview of Columns
| Column Name       | Description                                                                                       |
|-------------------|---------------------------------------------------------------------------------------------------|
| admin_unit        | The administrative region or unit (e.g., city, district, or municipality).                        |
| timestamp_local   | Local timestamp indicating the date and time of the observation or record.                         |
| rain_6h           | Total rainfall (mm) measured over the last 6 hours.                                               |
| rain_24h          | Total rainfall (mm) measured over the last 24 hours.                                              |
| wind_max_24h      | Maximum wind speed (km/h) recorded in the last 24 hours.                                           |
| temp_max_24h      | Maximum air temperature (°C) observed in the last 24 hours.                                       |
| precipitation     | Total precipitation (mm) during the measurement period.                                           |
| wind_gusts_10m    | Wind gust speed (km/h) measured at 10 meters above ground.                                         |
| temperature_2m    | Air temperature (°C) measured at 2 meters above ground.                                           |


In [11]:
weather_hourly_df = pd.read_csv('../Data/Weather/features_weather_hourly.csv')
weather_hourly_df.head()

,admin_unit,timestamp_local,rain_6h,rain_24h,wind_max_24h,temp_max_24h,precipitation,wind_gusts_10m,temperature_2m
0,Mombasa,2024-09-16 00:00:00,0.0,0.0,31.3,24.6,0.0,31.3,24.6
1,Mombasa,2024-09-16 01:00:00,0.0,0.0,31.3,24.6,0.0,30.2,24.5
2,Mombasa,2024-09-16 02:00:00,0.0,0.0,31.3,24.6,0.0,29.2,24.5
3,Mombasa,2024-09-16 03:00:00,0.0,0.0,31.3,24.6,0.0,28.1,24.5
4,Mombasa,2024-09-16 04:00:00,0.1,0.1,31.3,24.6,0.1,23.8,24.2


In [12]:
# Check shape
print(f"The weather dataset has {weather_hourly_df.shape[0]} rows and {weather_hourly_df.shape[1]} columns. ")

The weather dataset has 453456 rows and 9 columns. 


In [13]:
# Check the first 5 rows
weather_hourly_df.head()

,admin_unit,timestamp_local,rain_6h,rain_24h,wind_max_24h,temp_max_24h,precipitation,wind_gusts_10m,temperature_2m
0,Mombasa,2024-09-16 00:00:00,0.0,0.0,31.3,24.6,0.0,31.3,24.6
1,Mombasa,2024-09-16 01:00:00,0.0,0.0,31.3,24.6,0.0,30.2,24.5
2,Mombasa,2024-09-16 02:00:00,0.0,0.0,31.3,24.6,0.0,29.2,24.5
3,Mombasa,2024-09-16 03:00:00,0.0,0.0,31.3,24.6,0.0,28.1,24.5
4,Mombasa,2024-09-16 04:00:00,0.1,0.1,31.3,24.6,0.1,23.8,24.2


In [14]:
# Check the last 5 rows
weather_hourly_df.tail()

,admin_unit,timestamp_local,rain_6h,rain_24h,wind_max_24h,temp_max_24h,precipitation,wind_gusts_10m,temperature_2m
453451,Nairobi,2025-10-22 19:00:00,0.1,1.2,28.8,23.4,0.1,21.2,21.1
453452,Nairobi,2025-10-22 20:00:00,0.1,1.2,27.4,23.4,0.0,24.5,20.8
453453,Nairobi,2025-10-22 21:00:00,0.1,1.2,27.4,23.4,0.0,20.5,20.0
453454,Nairobi,2025-10-22 22:00:00,0.1,1.2,27.4,23.4,0.0,24.1,19.0
453455,Nairobi,2025-10-22 23:00:00,0.1,1.0,27.4,23.4,0.0,21.6,18.8


**Observation**  

The weather dataset is uniform from start to finish.

In [15]:
# Get metadata
weather_hourly_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 453456 entries, 0 to 453455
Data columns (total 9 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   admin_unit       453456 non-null  object 
 1   timestamp_local  453456 non-null  object 
 2   rain_6h          453456 non-null  float64
 3   rain_24h         453456 non-null  float64
 4   wind_max_24h     453456 non-null  float64
 5   temp_max_24h     453456 non-null  float64
 6   precipitation    453456 non-null  float64
 7   wind_gusts_10m   453456 non-null  float64
 8   temperature_2m   453456 non-null  float64
dtypes: float64(7), object(2)
memory usage: 31.1+ MB


**Observation** 

The weather dataset has 7 numerical columns and 2 categorical columns.

In [16]:
# statistical summary of numerical columns
weather_hourly_df.describe().T

,count,mean,std,min,25%,50%,75%,max
rain_6h,453456.0,0.695495,2.071994,0.0,0.0,0.0,0.4,103.2
rain_24h,453456.0,2.777101,4.928189,0.0,0.0,0.7,3.1,126.3
wind_max_24h,453456.0,35.957483,9.367650,3.2,29.2,34.6,41.8,77.4
temp_max_24h,453456.0,26.873013,4.331181,13.0,23.7,26.2,29.5,40.6
precipitation,453456.0,0.115936,0.620431,0.0,0.0,0.0,0.0,33.3
wind_gusts_10m,453456.0,22.161621,11.712860,0.7,13.0,20.5,29.5,77.4
temperature_2m,453456.0,21.216659,5.553944,7.4,16.8,20.9,25.1,40.6


## 2.4 Vegetation

| **Column**  | **Meaning**             | **Short Explanation**                                                                                  |
| ----------- | ----------------------- | ------------------------------------------------------------------------------------------------------ |
| `COUNTY`    | County name             | The Kenyan county where the NDVI (vegetation) data was measured.                                       |
| `date`      | Observation date        | The specific date (or composite period) for which the NDVI values were recorded.                       |
| `ndvi_mean` | Mean NDVI               | Average vegetation greenness over the county — higher = greener, healthier vegetation.                 |
| `ndvi_std`  | NDVI standard deviation | How much NDVI varies within the county — higher = more uneven vegetation (mix of green and dry areas). |
| `ndvi_min`  | Minimum NDVI            | Lowest NDVI value in the county — represents driest or least vegetated spot.                           |
| `ndvi_max`  | Maximum NDVI            | Highest NDVI value in the county — represents the greenest or most vegetated spot.                     |


In [17]:
# load dataset
vegetation_df = pd.read_csv('../Data/Vegetation Data/ndvi_timeseries.csv')

# preview the first 5 rows
vegetation_df.head()

,COUNTY,date,ndvi_mean,ndvi_std,ndvi_min,ndvi_max
0,Baringo,2024-01-01,0.482464,0.155480,-0.1993,0.9450
1,Baringo,2024-01-17,0.481681,0.158893,-0.1991,0.8946
2,Baringo,2024-02-02,0.437542,0.161304,-0.1998,0.9110
3,Baringo,2024-02-18,0.414909,0.150995,-0.1943,0.8851
4,Baringo,2024-03-05,0.445561,0.158213,-0.1998,0.9278


In [18]:
# preview the last 5 rows
vegetation_df.tail()

,COUNTY,date,ndvi_mean,ndvi_std,ndvi_min,ndvi_max
1922,West Pokot,2025-07-28,0.581488,0.158444,-0.1462,0.9220
1923,West Pokot,2025-08-13,0.615379,0.145057,-0.1960,0.9275
1924,West Pokot,2025-08-29,0.617688,0.120755,-0.1953,0.9209
1925,West Pokot,2025-09-14,0.598824,0.139189,-0.1979,0.9110
1926,West Pokot,2025-09-30,0.580714,0.140128,-0.1816,0.9122


In [19]:
# Check shape
print(f"The vegetation dataset has {vegetation_df.shape[0]} rows and {vegetation_df.shape[1]} columns.")

The vegetation dataset has 1927 rows and 6 columns.


In [20]:
# Get metadata
vegetation_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1927 entries, 0 to 1926
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   COUNTY     1927 non-null   object 
 1   date       1927 non-null   object 
 2   ndvi_mean  1927 non-null   float64
 3   ndvi_std   1927 non-null   float64
 4   ndvi_min   1927 non-null   float64
 5   ndvi_max   1927 non-null   float64
dtypes: float64(4), object(2)
memory usage: 90.5+ KB


**Observation**  
- The dataset has 4 numerical columns and 2 categorical columns.
- The date column should be changed to datetime datatype.
- We have no missing values

In [21]:
#check for unique values
for col in vegetation_df.columns:
    print(f'{col}: \n{vegetation_df[col].unique()} unique values\n')

COUNTY: 
['Baringo' 'Bomet' 'Bungoma' 'Busia' 'Embu' 'Garissa' 'Homa Bay' 'Isiolo'
 'Kajiado' 'Kakamega' 'Keiyo-Marakwet' 'Kericho' 'Kiambu' 'Kilifi'
 'Kirinyaga' 'Kisii' 'Kisumu' 'Kitui' 'Kwale' 'Laikipia' 'Lamu' 'Machakos'
 'Makueni' 'Mandera' 'Marsabit' 'Meru' 'Migori' 'Mombasa' "Murang'a"
 'Nairobi' 'Nakuru' 'Nandi' 'Narok' 'Nyamira' 'Nyandarua' 'Nyeri'
 'Samburu' 'Siaya' 'Taita Taveta' 'Tana River' 'Tharaka' 'Trans Nzoia'
 'Turkana' 'Uasin Gishu' 'Vihiga' 'Wajir' 'West Pokot'] unique values

date: 
['2024-01-01' '2024-01-17' '2024-02-02' '2024-02-18' '2024-03-05'
 '2024-03-21' '2024-04-06' '2024-04-22' '2024-05-08' '2024-05-24'
 '2024-06-09' '2024-06-25' '2024-07-11' '2024-07-27' '2024-08-12'
 '2024-08-28' '2024-09-13' '2024-09-29' '2024-10-15' '2024-10-31'
 '2024-11-16' '2024-12-02' '2024-12-18' '2025-01-01' '2025-01-17'
 '2025-02-02' '2025-02-18' '2025-03-06' '2025-03-22' '2025-04-07'
 '2025-04-23' '2025-05-09' '2025-05-25' '2025-06-10' '2025-06-26'
 '2025-07-12' '2025-07-28' '2

In [22]:
#get statistical summary of numerical variables
vegetation_df.describe().T

,count,mean,std,min,25%,50%,75%,max
ndvi_mean,1927.0,0.564311,0.156718,0.189737,0.451608,0.591627,0.696268,0.819885
ndvi_std,1927.0,0.125402,0.056588,0.028375,0.085830,0.124323,0.152528,0.331091
ndvi_min,1927.0,0.032850,0.222676,-0.200000,-0.194550,0.028300,0.143500,0.603700
ndvi_max,1927.0,0.893907,0.057047,0.528200,0.878850,0.904800,0.923800,0.998200


**Observations**

< 0 - water, clouds, or barren areas

0–0.2 - bare soil or sparse vegetation

0.2–0.5 - moderate vegetation

0.5-1 - dense, healthy vegetation

In [23]:
#get statistical summary of categorical variables
vegetation_df.describe(include="O").T

,count,unique,top,freq
COUNTY,1927,47,Baringo,41
date,1927,41,2024-01-01,47


## 2.5 Population Density Data Understanding
| **Column**                | **Meaning**                                                                       |
| ------------------------- | --------------------------------------------------------------------------------- |
| `county_code`             | Unique numeric identifier for each county (1–47).                                 |
| `county`                  | Name of the county (e.g., Nairobi, Kisii, etc.).                                  |
| `population`              | Total population of the county.                                                   |
| `male`                    | Total number of male residents.                                                   |
| `female`                  | Total number of female residents.                                                 |
| `intersex`                | Number of intersex persons recorded.                                              |
| `sex_ratio`               | Ratio of males per 100 females (e.g., 91.1 means 91 males for every 100 females). |
| `area_km2`                | Total land area of the county in square kilometers.                               |
| `density_per_km2`         | Population density — people per square kilometer.                                 |
| `number_of_households`    | Total number of households in the county.                                         |
| `avg_household_size`      | Average number of people per household.                                           |
| `percent_of_national_pop` | County’s share of Kenya’s total population (as a percentage).                     |


In [24]:
#load dataset
population_df = pd.read_csv('../Data/Population Density/kenya_counties_population_2019.csv')

In [25]:
# Check shape
print(f"The population dataset has {population_df.shape[0]} rows and {population_df.shape[1]} columns.")

The population dataset has 47 rows and 12 columns.


In [26]:
# Check head
population_df.head()

,county_code,county,population,male,female,intersex,sex_ratio,area_km2,density_per_km2,number_of_households,avg_household_size,percent_of_national_pop
0,1,Mombasa,1208333,610257,598046,30,102.0,219.9,5495,378422,3.1,2.54
1,2,Kwale,866820,425121,441681,18,96.3,8267.1,105,173176,5.0,1.82
2,3,Kilifi,1453787,704089,749673,25,93.9,12539.7,116,298472,4.8,3.06
3,4,Tana River,315943,158550,157391,2,100.7,37950.5,8,68242,4.6,0.66
4,5,Lamu,143920,76103,67813,4,112.2,6253.3,23,37963,3.7,0.30


In [27]:
# Check tail
population_df.tail()

,county_code,county,population,male,female,intersex,sex_ratio,area_km2,density_per_km2,number_of_households,avg_household_size,percent_of_national_pop
42,43,Homa Bay,1131950,539560,592367,23,91.1,3152.5,359,262036,4.3,2.38
43,44,Migori,1116436,536187,580214,35,92.4,2613.5,427,240168,4.6,2.35
44,45,Kisii,1266860,605784,661038,38,91.6,1323.0,958,308054,4.1,2.66
45,46,Nyamira,605576,290907,314656,13,92.5,897.3,675,150669,4.0,1.27
46,47,Nairobi,4397073,2192452,2204376,245,99.5,703.9,6247,1506888,2.9,9.24


**Observation**  

The population dataset is uniform from start to finish.


In [28]:
# Get metadata
population_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 47 entries, 0 to 46
Data columns (total 12 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   county_code              47 non-null     int64  
 1   county                   47 non-null     object 
 2   population               47 non-null     int64  
 3   male                     47 non-null     int64  
 4   female                   47 non-null     int64  
 5   intersex                 47 non-null     int64  
 6   sex_ratio                47 non-null     float64
 7   area_km2                 47 non-null     float64
 8   density_per_km2          47 non-null     int64  
 9   number_of_households     47 non-null     int64  
 10  avg_household_size       47 non-null     float64
 11  percent_of_national_pop  47 non-null     float64
dtypes: float64(4), int64(7), object(1)
memory usage: 4.5+ KB


**Observation**   
- There are 11 numerical columns and 1 categorical column.
- We don't have missing values.

In [29]:
#stastistical summary of numerical columns
population_df.describe().T

,count,mean,std,min,25%,50%,75%,max
county_code,47.0,2.400000e+01,13.711309,1.0,12.50,24.00,35.50,47.00
population,47.0,1.012006e+06,693242.707861,143920.0,609505.00,893681.00,1156723.50,4397073.00
male,47.0,5.010225e+05,342367.739953,76103.0,303109.50,450741.00,569992.00,2192452.00
female,47.0,5.109514e+05,351315.785111,67813.0,311291.00,448868.00,589759.00,2204376.00
intersex,47.0,3.242553e+01,38.490961,2.0,18.00,25.00,34.00,245.00
sex_ratio,47.0,9.922340e+01,6.193377,90.4,95.45,98.70,100.35,120.00
area_km2,47.0,1.235907e+04,17366.002689,219.9,2530.35,3325.00,14845.85,70944.10
density_per_km2,47.0,5.091489e+02,1175.429468,6.0,52.50,221.00,416.00,6247.00
number_of_households,47.0,2.583811e+05,234455.192186,37963.0,141956.00,204188.00,302844.00,1506888.00
avg_household_size,47.0,4.231915e+00,0.884186,2.9,3.50,4.30,4.65,6.90


In [30]:
# statistical summary of categorical columns
population_df.describe(include=object).T

,count,unique,top,freq
county,47,47,Mombasa,1


## 2.6 Power Stations Data Understanding

### Columns overview

The 'power_stations' dataset provides information on major power stations across Kenya.
It contains details such as station name, type, installed capacity, geographic coordinates,
operational status, and year of commissioning.

| Column Name | Description |
|--------------|-------------|
| `fid`, `objectid` | Unique identifiers for each power station. |
| `name` | Name of the power station. |
| `type` | Type of power generation (hydroelectric, geothermal, solar, wind, fossil fuels). |
| `capacity_m` | Installed power generation capacity in megawatts (MW). |
| `status` | Operational status (e.g., operational, planned, under construction). |
| `year` | Year of commissioning or development. |
| `longitude`, `latitude` | Geographic coordinates showing the station's location. |
| `remarks` | Additional notes or comments about the station. |

In [31]:
#load dataset
power_stations_df = pd.read_csv('../Data/Power Grid and Infrastructure/power_stations.csv')

#preview first five rows
power_stations_df.head()

,FID,OBJECTID,Type,Name,Capacity_M,Coordinate,X_Coord,Y_Coord,Status,Year,Remarks,longitude,latitude
0,0,1,Geothermal,Olkaria I Geothermal Power Station,185.0,0°53′36″S 36°18′30″E,36.308333,-0.893333,Operational,,,200410.145227,9.901150e+06
1,1,2,Geothermal,Olkaria II Geothermal Power Station,105.0,0°51′49″S 36°18′00″E,36.300000,-0.863611,Operational,,,199479.554176,9.904438e+06
2,2,3,Geothermal,Olkaria III Geothermal Power Station,110.0,0°52′30″S 36°18′00″E,36.300000,-0.875000,Operational,,,199480.455853,9.903178e+06
3,3,4,Geothermal,Olkaria IV Geothermal Power Station,140.0,0°51′49″S 36°18′00″E,36.300000,-0.863611,Operational,,,199479.554176,9.904438e+06
4,4,5,Geothermal,Olkaria V Geothermal Power Station,140.0,0°51′49″S 36°18′00″E,36.300000,-0.863611,In development,,,199479.554176,9.904438e+06


In [32]:
#check the last 5 rows of the dataset
power_stations_df.tail()

,FID,OBJECTID,Type,Name,Capacity_M,Coordinate,X_Coord,Y_Coord,Status,Year,Remarks,longitude,latitude
35,35,36,Hydroelectric,Sagana Power Station,1.5,,0.0,0.0,Commissioned,1956,Reservoir,283139.535288,9.947438e+06
36,36,37,Hydroelectric,Kiambere Power Station,165.0,,0.0,0.0,Commissioned,1988,Reservoir,376325.417285,9.933699e+06
37,37,38,Hydroelectric,Gogo Power Station,2.0,,0.0,0.0,Commissioned,1957,Reservoir,-18157.192280,9.899145e+06
38,38,39,Hydroelectric,Mesco Power Station,0.4,,0.0,0.0,Commissioned,1930,Reservoir,293957.496537,9.916539e+06
39,39,40,Hydroelectric,Sosiani Power Station,0.4,,0.0,0.0,Commissioned,1955,Reservoir,74390.534517,1.006056e+07


**Observations**: The dataset has uniformity from top to bottom

In [33]:
#check for the data info
power_stations_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40 entries, 0 to 39
Data columns (total 13 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   FID         40 non-null     int64  
 1   OBJECTID    40 non-null     int64  
 2   Type        40 non-null     object 
 3   Name        40 non-null     object 
 4   Capacity_M  40 non-null     float64
 5   Coordinate  40 non-null     object 
 6   X_Coord     40 non-null     float64
 7   Y_Coord     40 non-null     float64
 8   Status      40 non-null     object 
 9   Year        40 non-null     object 
 10  Remarks     40 non-null     object 
 11  longitude   40 non-null     float64
 12  latitude    40 non-null     float64
dtypes: float64(5), int64(2), object(6)
memory usage: 4.2+ KB


**Observations**
- The dataset contains 40 rows and 13 columns. Each row represents a power station in Kenya.  
- All columns have complete data with no missing values.
- The dataset includes both numeric and text values.  
- Columns such as Type, Status, and Remarks contain descriptive information about each power station. 
- Numerical fields like Capacity_ , X_Coor , Y_Coor , longitud , and latitud  describe the station’s capacity and location.  
- The Year column is stored as text and may need conversion to numeric format for analysis.  

In [34]:
#check for the datasets shape
print(f"The dataset has {power_stations_df.shape[0]} rows and {power_stations_df.shape[1]} columns.")

The dataset has 40 rows and 13 columns.


In [35]:
#check for missing values
print(power_stations_df.isnull().sum())

FID           0
OBJECTID      0
Type          0
Name          0
Capacity_M    0
Coordinate    0
X_Coord       0
Y_Coord       0
Status        0
Year          0
Remarks       0
longitude     0
latitude      0
dtype: int64


The dataset has 0 missing values

In [36]:
#check for the duplicated values
print(power_stations_df.duplicated().sum())

0


The dataset has zero duplicate values.

## 2.7  Athi River

### Site Meter — Column Dictionary

| Column                    | Description |
|---------------------------|-------------|
| **Time**                  | Local timestamp (Africa/Nairobi) of the measurement record. Will be standardized to `timestamp_local` and resampled to hourly. |
| **Apparent power(+)(kVA)**| Apparent power drawn by the site in **kVA** (S). Used with kW to derive/validate power factor. |
| **Active power(+)(kW)**   | Real/active power consumed in **kW** (P). Primary indicator of load/energy use. |
| **Power factor(%)**       | Aggregate power factor (P/S) on a **0–1** scale (often shown as 0–1; some exports label “%”). Values outside [0,1] are invalid and will be fixed/derived. |
| **A phase power factor(%)** | Phase-A power factor (0–1). |
| **B phase power factor(%)** | Phase-B power factor (0–1). |
| **C phase power factor(%)** | Phase-C power factor (0–1). |
| **A phase current(A)**    | Phase-A line current in **Amperes**. Used to detect phase imbalance and stress peaks. |
| **B phase current(A)**    | Phase-B line current in **Amperes**. |
| **C phase current(A)**    | Phase-C line current in **Amperes**. |
| **A phase voltage(V)**    | Phase-A line voltage in **Volts** (nominal ≈ 230 V). Used for sag/over-voltage flags. |
| **B phase voltage(V)**    | Phase-B line voltage in **Volts**. |
| **C phase voltage(V)**    | Phase-C line voltage in **Volts**. |

In [37]:
#loading dataset
athi_df = pd.read_excel('../Data/Energy Consumption/Athi River.xls', engine='xlrd')
#preview first 5 rows
athi_df.head()

,Time,Apparent power(+)(kVA),Active power(+)(kW),Power factor(%),A phase power factor(%),B phase power factor(%),C phase power factor(%),A phase current(A),B phase current(A),C phase current(A),A phase voltage(V),B phase voltage(V),C phase voltage(V)
0,2025-10-26 02:00:00,1.8000,1.35,0.750,0.782,0.628,0.924,2.4,5.1,0.6,231.6,231.3,229.0
1,2025-10-26 01:00:00,1.8008,1.32,0.733,0.773,0.630,0.921,2.4,5.1,0.6,230.6,230.3,228.4
2,2025-10-26 12:00:00,1.2300,1.23,1.000,0.797,0.648,0.885,2.1,4.8,0.6,225.5,225.2,223.3
3,2025-10-26 11:00:00,1.8000,1.26,0.700,0.789,0.643,0.885,2.1,4.8,0.6,226.9,226.3,224.3
4,2025-10-26 10:00:00,1.8017,1.29,0.716,0.781,0.638,0.877,2.1,4.8,0.6,229.3,229.2,226.8


In [38]:
#previewing last 5 rows
athi_df.tail()

,Time,Apparent power(+)(kVA),Active power(+)(kW),Power factor(%),A phase power factor(%),B phase power factor(%),C phase power factor(%),A phase current(A),B phase current(A),C phase current(A),A phase voltage(V),B phase voltage(V),C phase voltage(V)
39170,2020-10-01 04:00:00,15.7500,15.75,1.000,0.930,0.965,0.979,11.4,18.0,39.3,235.0,235.5,235.3
39171,2020-10-01 03:00:00,16.4700,16.47,1.000,0.923,0.948,0.977,12.0,19.2,40.5,236.3,236.7,236.6
39172,2020-10-01 02:00:00,20.9324,20.43,0.976,0.919,0.925,0.965,18.3,26.1,46.5,235.0,235.5,235.3
39173,2020-10-01 01:00:00,20.4086,19.98,0.979,0.906,0.914,0.961,17.4,25.5,45.9,237.2,237.8,237.5
39174,2020-10-01 12:00:00,17.9506,17.43,0.971,0.851,0.885,0.955,14.4,22.2,42.9,235.5,236.1,235.7


**Observations**: The data is uniform from top to bottom

In [39]:
#checking the shape of the dataset
print(f"The dataset has {athi_df.shape[0]} records and {athi_df.shape[1]} features")

The dataset has 39175 records and 13 features


In [40]:
#checking the dataset information
athi_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39175 entries, 0 to 39174
Data columns (total 13 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Time                     39175 non-null  object 
 1   Apparent power(+)(kVA)   38942 non-null  float64
 2   Active power(+)(kW)      38991 non-null  float64
 3   Power factor(%)          39097 non-null  float64
 4   A phase power factor(%)  39019 non-null  float64
 5   B phase power factor(%)  39019 non-null  float64
 6   C phase power factor(%)  39019 non-null  float64
 7   A phase current(A)       39019 non-null  float64
 8   B phase current(A)       39019 non-null  float64
 9   C phase current(A)       39019 non-null  float64
 10  A phase voltage(V)       39019 non-null  float64
 11  B phase voltage(V)       39019 non-null  float64
 12  C phase voltage(V)       39019 non-null  float64
dtypes: float64(12), object(1)
memory usage: 3.9+ MB


**Observations**
1. The columns are of float data type except for the time which is an object data type that could be changed to datetime.
2. The time column is the only column without missing values. The rest will need to be handled during cleaning.

In [41]:
#checking statistical summary
athi_df.describe(include="all").T

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
Time,39175,20189,2025-10-26 02:00:00,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Apparent power(+)(kVA),38942.0,NaN,NaN,NaN,37.491084,140.324876,0.0,8.24755,22.19465,51.06045,17394.0
Active power(+)(kW),38991.0,NaN,NaN,NaN,31.006491,31.613814,0.0,7.2,18.93,48.24,223.71
Power factor(%),39097.0,NaN,NaN,NaN,0.904688,0.188196,0.0,0.92,0.979,1.0,1.0
A phase power factor(%),39019.0,NaN,NaN,NaN,0.833157,0.219106,0.011,0.8145,0.934,0.959,1.0
B phase power factor(%),39019.0,NaN,NaN,NaN,0.82129,0.192435,0.0,0.729,0.913,0.956,1.0
C phase power factor(%),39019.0,NaN,NaN,NaN,0.74773,0.237614,0.0,0.614,0.862,0.926,1.0
A phase current(A),39019.0,NaN,NaN,NaN,46.77685,43.988058,0.0,14.7,30.6,66.9,337.5
B phase current(A),39019.0,NaN,NaN,NaN,58.229985,52.356487,0.0,14.4,43.5,88.8,357.0
C phase current(A),39019.0,NaN,NaN,NaN,49.597429,47.573146,0.0,13.5,35.4,72.6,348.6


**Observations**
- Row counts vary slightly per column; some columns have missing values.
- Many columns have min = 0 these are likely true zeros (plant off) or missing encoded as 0. We will verify this via time plots.
- The data comprises of rows from 01/10/2020 to 26/10/2025
- The 17,394 kVA apparent power is an implausible outlier that could be due to a meter glitch or unit error and should be handled

## 2.8 Garissa

### Site Meter — Column Dictionary

| Column                    | Description |
|---------------------------|-------------|
| **Time**                  | Local timestamp (Africa/Nairobi) of the measurement record. Will be standardized to `timestamp_local` and resampled to hourly. |
| **Apparent power(+)(kVA)**| Apparent power drawn by the site in **kVA** (S). Used with kW to derive/validate power factor. |
| **Active power(+)(kW)**   | Real/active power consumed in **kW** (P). Primary indicator of load/energy use. |
| **Power factor(%)**       | Aggregate power factor (P/S) on a **0–1** scale (often shown as 0–1; some exports label “%”). Values outside [0,1] are invalid and will be fixed/derived. |
| **A phase power factor(%)** | Phase-A power factor (0–1). |
| **B phase power factor(%)** | Phase-B power factor (0–1). |
| **C phase power factor(%)** | Phase-C power factor (0–1). |
| **A phase current(A)**    | Phase-A line current in **Amperes**. Used to detect phase imbalance and stress peaks. |
| **B phase current(A)**    | Phase-B line current in **Amperes**. |
| **C phase current(A)**    | Phase-C line current in **Amperes**. |
| **A phase voltage(V)**    | Phase-A line voltage in **Volts** (nominal ≈ 230 V). Used for sag/over-voltage flags. |
| **B phase voltage(V)**    | Phase-B line voltage in **Volts**. |
| **C phase voltage(V)**    | Phase-C line voltage in **Volts**. |

In [42]:
# loading the dataset
garissa_df = pd.read_excel("../Data/Energy Consumption/Garissa.xls", engine='xlrd')

#previewing first 5 rows
garissa_df.head()

,Time,Apparent power(+)(kVA),Active power(+)(kW),Power factor(%),A phase power factor(%),B phase power factor(%),C phase power factor(%),A phase current(A),B phase current(A),C phase current(A),A phase voltage(V),B phase voltage(V),C phase voltage(V)
0,2025-10-26 03:00:00,27.0343,25.25,0.934,0.867,0.891,0.872,44.5,47.7,26.9,236.5,232.0,226.9
1,2025-10-26 02:00:00,33.4611,30.55,0.913,0.937,0.916,0.962,63.0,45.4,31.4,233.2,232.3,227.0
2,2025-10-26 01:00:00,16.1799,15.29,0.945,0.613,0.743,0.966,21.8,31.4,39.7,240.3,234.8,222.4
3,2025-10-26 12:00:00,24.8276,3.60,0.145,0.319,0.485,0.404,22.0,24.1,10.4,240.8,236.9,229.5
4,2025-10-26 11:00:00,36.8539,3.28,0.089,0.297,0.266,0.619,23.4,24.5,13.3,241.2,238.2,230.5


In [43]:
#previewing last 5 rows
garissa_df.tail()

,Time,Apparent power(+)(kVA),Active power(+)(kW),Power factor(%),A phase power factor(%),B phase power factor(%),C phase power factor(%),A phase current(A),B phase current(A),C phase current(A),A phase voltage(V),B phase voltage(V),C phase voltage(V)
42211,2020-10-01 04:00:00,24.2199,23.13,0.955,0.922,0.951,0.969,19.8,35.8,43.8,245.6,245.7,240.6
42212,2020-10-01 03:00:00,22.6000,21.47,0.950,0.928,0.933,0.955,19.9,36.1,37.8,244.4,244.7,241.3
42213,2020-10-01 02:00:00,26.9295,25.61,0.951,0.914,0.937,0.949,31.4,37.5,44.3,241.1,243.9,239.2
42214,2020-10-01 01:00:00,25.6144,24.18,0.944,0.928,0.950,0.951,29.7,35.7,40.7,240.6,243.6,239.3
42215,2020-10-01 12:00:00,26.5263,25.20,0.950,0.954,0.947,0.950,25.1,36.9,48.9,240.3,242.2,235.9


**Observations**: The data is uniform from top to bottom

In [44]:
#checking the shape of the dataset
print(f"The dataset has {garissa_df.shape[0]} records and {garissa_df.shape[1]} features")

The dataset has 42216 records and 13 features


In [45]:
#checking the dataset information
garissa_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42216 entries, 0 to 42215
Data columns (total 13 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Time                     42216 non-null  object 
 1   Apparent power(+)(kVA)   42053 non-null  float64
 2   Active power(+)(kW)      42109 non-null  float64
 3   Power factor(%)          42152 non-null  float64
 4   A phase power factor(%)  42114 non-null  float64
 5   B phase power factor(%)  42114 non-null  float64
 6   C phase power factor(%)  42114 non-null  float64
 7   A phase current(A)       42114 non-null  float64
 8   B phase current(A)       42114 non-null  float64
 9   C phase current(A)       42114 non-null  float64
 10  A phase voltage(V)       42114 non-null  float64
 11  B phase voltage(V)       42114 non-null  float64
 12  C phase voltage(V)       42114 non-null  float64
dtypes: float64(12), object(1)
memory usage: 4.2+ MB


**Observations**
1. The columns are of float data type except for the time which is an object data type that could be changed to datetime.
2. The time column is the only column without missing values. The rest will need to be handled during cleaning.

In [46]:
#checking statistical summary
garissa_df.describe(include="all").T

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
Time,42216,21607,2025-10-26 03:00:00,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Apparent power(+)(kVA),42053.0,NaN,NaN,NaN,31.138051,36.913445,0.0,20.6814,29.1224,39.2893,5795.0
Active power(+)(kW),42109.0,NaN,NaN,NaN,29.141639,14.890137,0.0,19.7,28.1,38.09,107.19
Power factor(%),42152.0,NaN,NaN,NaN,0.953864,0.121907,0.0,0.965,0.982,0.994,1.0
A phase power factor(%),42114.0,NaN,NaN,NaN,0.925276,0.111453,0.0,0.922,0.959,0.979,1.0
B phase power factor(%),42114.0,NaN,NaN,NaN,0.909306,0.123827,0.0,0.911,0.945,0.967,1.0
C phase power factor(%),42114.0,NaN,NaN,NaN,0.958056,0.084231,0.0,0.964,0.977,0.986,1.0
A phase current(A),42114.0,NaN,NaN,NaN,39.639455,24.967062,0.0,21.6,34.7,52.6,182.1
B phase current(A),42114.0,NaN,NaN,NaN,45.439953,25.471745,0.0,26.9,42.3,60.6,192.2
C phase current(A),42114.0,NaN,NaN,NaN,46.949167,22.420647,0.0,33.3,45.5,60.1,174.9


**Observations**
- Row counts vary slightly per column; some columns have missing values.
- Row counts are 42216 but time has 21607 unique stamps which suggests presence of duplicates.
- Many columns have min = 0 these are likely true zeros (plant off) or missing encoded as 0. We will verify this via time plots.
- The data comprises of rows from 01/10/2020 to 26/10/2025
- The 5794 kVA apparent power is an implausible outlier that could be due to a meter glitch or unit error and should be handled
- Presence of voltage anomalies exceeding 260V range which is physically impossible and should be handled.

## 2.9 Energy Consumption, Kaggle

### Building Energy Dataset — Column Dictionary

| Column              | Description |
|---------------------|-------------|
| **RoomArea**        | Floor area of the space (units unspecified in source; treat as relative size). |
| **NumberofAppliances** | Count of electrical appliances in the space. |
| **Outside Temperature** | Outdoor air temperature (°C) at the time/period of measurement. |
| **InsulationThickness** | Thermal insulation thickness of the building envelope (units unspecified; relative measure). |
| **BuildingType**    | Category of the building (e.g., Commercial, Residential). |
| **HVACSystem**      | Type of HVAC installed (e.g., Window AC, Split, Central). |
| **AverageTemperature** | Indoor average air temperature (°C) over the measurement period. |
| **EnergyConsumption** | Energy used over the measurement period (units/timebase not specified in source—document assumption). |


In [47]:
# loading the dataset
consumption_df = pd.read_csv("../Data/Energy Consumption/energy_consumption.csv")

#previewing first 5 rows
consumption_df.head()

,RoomArea,NumberofAppliances,Outside Temperature,InsulationThickness,BuildingType,HVACSystem,AverageTemperature,EnergyConsumption
0,302,5,-9.638784,7.227564,Residential,Window AC,-8.444161,1828.820920
1,470,17,13.536505,1.086168,Residential,Window AC,15.267517,2307.834235
2,306,17,13.948559,3.590366,Commercial,Window AC,12.029111,1591.374161
3,271,18,5.866391,1.191942,Residential,Split AC,4.481526,1463.371403
4,388,12,-0.518886,3.750820,Residential,Central AC,1.025799,2198.909856


In [48]:
#previewing last 5 rows
consumption_df.tail()

,RoomArea,NumberofAppliances,Outside Temperature,InsulationThickness,BuildingType,HVACSystem,AverageTemperature,EnergyConsumption
995,414,13,5.369585,6.234870,Residential,Window AC,5.873804,2163.915879
996,410,16,8.004934,2.054257,Residential,Window AC,6.306049,2113.570960
997,231,6,13.170654,2.520616,Residential,Window AC,11.620242,969.943740
998,357,7,20.374041,4.247931,Commercial,Central AC,18.571277,1602.923718
999,484,9,-6.045055,2.848030,Commercial,Central AC,-7.015138,2643.004907


**Observations**: The data is uniform from top to bottom

In [49]:
#checking the shape of the dataset
print(f"The dataset has {consumption_df.shape[0]} records and {consumption_df.shape[1]} features")

The dataset has 1000 records and 8 features


In [50]:
#checking the dataset information
consumption_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 8 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   RoomArea             1000 non-null   int64  
 1   NumberofAppliances   1000 non-null   int64  
 2   Outside Temperature  1000 non-null   float64
 3   InsulationThickness  1000 non-null   float64
 4   BuildingType         1000 non-null   object 
 5   HVACSystem           1000 non-null   object 
 6   AverageTemperature   1000 non-null   float64
 7   EnergyConsumption    1000 non-null   float64
dtypes: float64(4), int64(2), object(2)
memory usage: 62.6+ KB


**Observations**: 
The columns have both numerical(int and float type) and categorical data. There are no missing values.

In [51]:
#checking statistical summary of categorical variables
consumption_df.describe(include="O").T

,count,unique,top,freq
BuildingType,1000,2,Commercial,507
HVACSystem,1000,3,Window AC,349


**Observations**

1. There are two building types, residential and commercial whereby the most frequent type is commercial.
2. There are three HVAC system types with WindowAC topping the types.

In [52]:
#checking statistical summary of numerical variables
consumption_df.describe().T

,count,mean,std,min,25%,50%,75%,max
RoomArea,1000.0,349.724000,85.444488,200.000000,278.500000,349.500000,423.250000,499.000000
NumberofAppliances,1000.0,11.999000,4.358669,5.000000,8.000000,12.000000,16.000000,19.000000
Outside Temperature,1000.0,9.719271,11.459586,-9.992464,-0.537895,9.829999,19.674842,29.883126
InsulationThickness,1000.0,4.592226,1.991828,1.003099,2.900386,4.687266,6.314432,7.997997
AverageTemperature,1000.0,9.712394,11.473614,-11.510723,-0.442051,9.880269,19.586637,31.492076
EnergyConsumption,1000.0,1813.907085,470.603884,731.680918,1453.442638,1803.590371,2168.074502,2920.190528


**Observations**
- Negative indoor temperatures are impossible hence should be imputed during cleaning.
- Units of the variables to be confirmed

## 2.10 Primary Substations Data Understanding

### Column Overview

| Column Name | Data Type | Description |
|--------------|------------|-------------|
| FID | Integer | Unique record identifier |
| OBJECTID | Integer | Object identification number |
| RCC1 | Object | Regional Control Centre name |
| County2 | Object | County where the substation is located |
| Branch3 | Object | KPLC operational branch |
| Name8 | Object | Substation name |
| Name_IMS11 | Object | Internal system name for the substation |
| Ownershi24 | Object | Ownership type (mainly KPLC) |
| Availabi39 | Object | Availability status (YES/NO) |
| Street40 | Object | Street name or physical address |
| Location41 | Object | Area or location name |
| Outgoing48 | Object | Outgoing voltage level (e.g., 11kV) |
| Manned50 | Object | Indicates if the substation is manned (YES/NO) |
| Availabi51 | Object | Additional availability field |
| Earthing56 | Object | Indicates earthing system status (YES/NO) |
| Remote_T57 | Object | Indicates if the substation is remotely controlled (YES/NO) |
| longitude | Float | Easting coordinate (UTM format) |
| latitude | Float | Northing coordinate (UTM format) |


In [53]:
#load the dataset
substations_df = pd.read_csv('../Data/Power Grid and Infrastructure/primary_substations.csv')
#read the first few rows of the dataset
substations_df.head()

,FID,OBJECTID,RCC1,County2,Branch3,Name8,Name_IMS11,Ownershi24,Availabi39,Street40,Location41,Outgoing48,Manned50,Availabi51,Earthing56,Remote_T57,longitude,latitude
0,0,1,Juja-Nairobi North,Nairobi North,Ngara - Eastleigh,PARKLANDS,PARKLANDS ( 66 / 11 KV),KPLC,YES,LIMURU R.D,PARKLANDS,11,YES,YES,YES,YES,257504.944880,9.859384e+06
1,1,2,Juja-Nairobi North,Nairobi North,City Centre,JEEVANJEE,JEEVANJEE ( 66 / 11 KV),KPLC,YES,RACECOURSE RD,CITY CENTRE,11,YES,YES,YES,YES,258513.019750,9.857610e+06
2,2,3,Juja-Nairobi North,Nairobi North,City Centre,CATHEDRAL,CATHEDRAL ( 66 / 11 KV),KPLC,YES,UHURU HIGHWAY,CITY CENTRE,11,NO,YES,YES,YES,257096.063665,9.857859e+06
3,3,4,Juja-Nairobi North,Nairobi North,City Centre,UNIVERSITY WAY,UNIVERSITY WAY ( 11 / 11 KV),KPLC,YES,UNIVERSITY WAY,CTY CENTRE,11,NO,NO,YES,NO,256961.749868,9.858320e+06
4,4,5,Juja-Nairobi North,Nairobi North,City Centre,DARA,DARA ( 11 / 11 KV),KPLC,NO,LUSAKA RD.,INDUSTRIAL AREA,11,NO,NO,YES,NO,258743.239612,9.856004e+06


In [54]:
#check the last five rows of the dataset
substations_df.tail()

,FID,OBJECTID,RCC1,County2,Branch3,Name8,Name_IMS11,Ownershi24,Availabi39,Street40,Location41,Outgoing48,Manned50,Availabi51,Earthing56,Remote_T57,longitude,latitude
259,259,260,Kiganjo North Eastern,Mandera,Mandera,RHAMU POWER STATION,RHAMU POWER STATION ( NONE /,KPLC,,,RHAMU MKT MANDERA,33,YES,,YES,,744778.103727,1.043395e+07
260,260,261,Lanet,Nyandarua,Olkalou,WANJOHI,WANJOHI ( 33 / 11 KV),KPLC,,,WANJOHI JUNCTION,11,NO,,YES,,221509.482478,9.964014e+06
261,261,262,Juja-Nairobi North,Nairobi North,Ngara - Eastleigh,RUARAKA 66/11KV,RUARAKA 66/11KV ( 66 / 11 KV),KPLC,,,RUARAKA,11,YES,,YES,,261890.271690,9.860760e+06
262,262,263,Juja-Nairobi South,Machakos,Tala,TALA 66KV/11KV,TALA 66KV/11KV ( 66 / 11 KV),KPLC,,,NGULUNI,11,YES,,YES,,308071.380223,9.857969e+06
263,263,264,Kiganjo North Eastern,Kiambu,Thika,JKUAT 66KV/11KV,JKUAT 66KV/11KV ( 66 / 11 KV),KPLC,,,JKUAT,11,YES,YES,YES,,277788.330776,9.879771e+06


**Observation:** The dataset values are uniform from top to bottom.

In [55]:
# check the dataset info
substations_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 264 entries, 0 to 263
Data columns (total 18 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   FID         264 non-null    int64  
 1   OBJECTID    264 non-null    int64  
 2   RCC1        264 non-null    object 
 3   County2     264 non-null    object 
 4   Branch3     264 non-null    object 
 5   Name8       264 non-null    object 
 6   Name_IMS11  264 non-null    object 
 7   Ownershi24  264 non-null    object 
 8   Availabi39  264 non-null    object 
 9   Street40    264 non-null    object 
 10  Location41  264 non-null    object 
 11  Outgoing48  264 non-null    object 
 12  Manned50    264 non-null    object 
 13  Availabi51  264 non-null    object 
 14  Earthing56  264 non-null    object 
 15  Remote_T57  264 non-null    object 
 16  longitude   264 non-null    float64
 17  latitude    264 non-null    float64
dtypes: float64(2), int64(2), object(14)
memory usage: 37.2+ KB


**Observation:**

- The dataset consists of 264 rows and 18 columns.
- It has 2 numerical columns (longitude and latitude), 14 text columns and 2 integer identifier columns (FID, OBJECTID).
- Column naming is inconsistent with mixed uppercase, lowercase, and numbers at the end , shows that column name standardization is needed.


In [56]:
#check for the dataset shape
print(f"The dataset has{substations_df.shape[0]} rows and {substations_df.shape[1]} columns.")

The dataset has264 rows and 18 columns.


In [57]:
#check for missing values
substations_df.isnull().sum()

FID           0
OBJECTID      0
RCC1          0
County2       0
Branch3       0
Name8         0
Name_IMS11    0
Ownershi24    0
Availabi39    0
Street40      0
Location41    0
Outgoing48    0
Manned50      0
Availabi51    0
Earthing56    0
Remote_T57    0
longitude     0
latitude      0
dtype: int64

**Observation**: The columns have 0 missing values. This shows the dataset is complete with no null values.

In [58]:
#check on the dataset statistical summary
substations_df.describe()

,FID,OBJECTID,longitude,latitude
count,264.000000,264.000000,264.000000,2.640000e+02
mean,131.500000,132.500000,265009.938581,9.894892e+06
std,76.354437,76.354437,203588.346468,1.796403e+05
min,0.000000,1.000000,-658779.677285,9.491473e+06
25%,65.750000,66.750000,114504.018069,9.850965e+06
50%,131.500000,132.500000,262007.938453,9.901779e+06
75%,197.250000,198.250000,335493.072483,9.994135e+06
max,263.000000,264.000000,817297.836855,1.046139e+07


In [59]:
#check for duplicate values
substations_df.duplicated().sum()

0

**Observations**: There are 0 duplicated values. Each row is unique

In [60]:
#check for the data types
substations_df.dtypes

FID             int64
OBJECTID        int64
RCC1           object
County2        object
Branch3        object
Name8          object
Name_IMS11     object
Ownershi24     object
Availabi39     object
Street40       object
Location41     object
Outgoing48     object
Manned50       object
Availabi51     object
Earthing56     object
Remote_T57     object
longitude     float64
latitude      float64
dtype: object

## 2.11 Transformers Data Understanding

### Grid Infrastructure (EnergyInfo extract) — Column Dictionary

| Column        | Description |
|---------------|-------------|
| **FID**       | Internal feature ID from the GIS export; not meaningful for analysis (can drop). |
| **OBJECTID**  | GIS object identifier; unique within the source layer (can drop after ingest). |
| **RCC1**      | Region / control center code or name assigned by the utility (e.g., operating region). |
| **County2**   | County in which the asset/feature is located (admin unit). |
| **Branch3**   | Utility branch / service office responsible for the asset. |
| **SubStat19** | Substation name/code associated with the record (free-text). |
| **DCS_Catie5**| Asset category/class as recorded in DCS (e.g., transformer, switch, line segment). |
| **Modifyin25**| Last modification date/user or flag from the source system (audit trail). |
| **Origin_02** | Data origin/source tag (survey, digitized, imported). |
| **Feeder_028**| Feeder name/ID to which the asset is connected. |
| **Substatio29**| Alternate/legacy substation field (duplicate of **SubStat19**)—standardize to a single substation column. |
| **HT_Isolsa12**| High-tension isolator/switch identifier (if applicable). |
| **Road_Str38**| Road/Street reference string near the asset (context for location). |
| **Physical39**| Physical address/landmark or additional location descriptor. |
| **Road**      | Road name (cleaner version of the street field where available). |
| **latitude**  | Latitude (WGS84) of the feature. |
| **longitude** | Longitude (WGS84) of the feature. |

In [61]:
# Load and read dataset
transformers_df = pd.read_csv("../Data/Power Grid and Infrastructure/distribution_transformers.csv")

In [62]:
# Check shape
print(f"The distribution transformers has {transformers_df.shape[0]} rows and {transformers_df.shape[1]} columns.")

The distribution transformers has 57360 rows and 17 columns.


In [63]:
# Check head
transformers_df.head()

,FID,OBJECTID,RCC1,County2,Branch3,Substati9,DCS_Cate15,Modifyin25,Origin_o27,Feeder_o28,Substati29,HT_Isola32,Road_Str38,Physical39,Road,longitude,latitude
0,0,1,Juja-Nairobi North,Nairobi North,Parklands - Kitsuru,H-POLE STRUCTURE 50-315 KVA 11/0.433 K.V. (E /...,( CODE 2 ) - SUBSTATION,1020_POT_INS,PARKLANDS,WELBECK,,FUSES,1st Parklands,1st Parklands-Ita road juncti,1ST PARKLANDS AVENUE,257569.268308,9.859955e+06
1,1,2,Juja-Nairobi North,Nairobi North,Parklands - Kitsuru,UNKNOWN,UNKNOWN,1020_POT_INS,PARKLANDS,WELBECK,,FUSES,ita road,next to starbright apartments,OFF 1ST EVE.PARKLANDS,257648.529430,9.860077e+06
2,2,3,Juja-Nairobi North,Nairobi North,Parklands - Kitsuru,H-POLE CONCRETE STRUCTURE 50-315KVA 11/0.433KV...,( CODE 3 ) - L.V,1020_POT_INS,PARKLANDS,WELBECK,OUTDOOR,FUSES,1st parklands avenue,Next to new executive ladies,1ST PARKLANDS AVENUE,257313.317356,9.860059e+06
3,3,4,Juja-Nairobi North,Nairobi North,Parklands - Kitsuru,H-POLE STRUCTURE 50-315 KVA 11/0.433 K.V. (E /...,( CODE 2 ) - SUBSTATION,1020_POT_INS,PARKLANDS,WELBECK,,FUSES,1st parklands avenue,opposite kingsbury court,1ST PARKLANDS AVENUE,257141.827903,9.860163e+06
4,4,5,Juja-Nairobi North,Nairobi North,Parklands - Kitsuru,H-POLE-WOODEN STRUCTURE 50-315 KVA 11/0.433 K....,( CODE 1 ) - H. V./M.V,1020_POT_INS,PARKLANDS,WELBECK,OUTDOOR,TAPLINS,1st parkland rd,next to kings bury court,1ST PARKLANDS AVENUE,257014.618676,9.860227e+06


In [64]:
# Check tail
transformers_df.tail()

,FID,OBJECTID,RCC1,County2,Branch3,Substati9,DCS_Cate15,Modifyin25,Origin_o27,Feeder_o28,Substati29,HT_Isola32,Road_Str38,Physical39,Road,longitude,latitude
57355,57355,57356,Kiganjo Mt kenya,Meru,Meru,SINGLE-POLE STRUCTURE 5-25 KVA 11/0.433 K.V.(E...,( CODE 2 ) - SUBSTATION,,KIIRUA,KIIRUA EX KIIRUA 33/11KV,OUTDOOR,,,KIIRUA,SYSTEM REINFORCEMENT S/S 26871,337037.613076,1.001698e+07
57356,57356,57357,Lanet,Nakuru,Nakuru Environs,H-POLE-WOODEN STRUCTURE 50-315 KVA 11/0.433 K....,( CODE 2 ) - SUBSTATION,,NAKURU DEPOT,NJORO 11KV EX NAKURU,OUTDOOR,,,MAUCHE,LELECHONIK PRIMARY,165715.890550,9.942714e+06
57357,57357,57358,Juja-Nairobi West,Kajiado,Kiserian,H-POLE STRUCTURE 50-315 KVA 11/0.433 K.V. (E /...,( CODE 2 ) - SUBSTATION,,NGONG,NGONG VET EX NGONG,,,,KERARAPON,NGONG,239103.608449,9.854033e+06
57358,57358,57359,Kiganjo North Eastern,Kiambu,Thika,H-POLE CONCRETE STRUCTURE 50-315KVA 11/0.433KV...,( CODE 1 ) - H. V./M.V,,THIKA,THIKA TOWN,OUTDOOR,,,SECTION 9,SECTION 9,284667.636642,9.884919e+06
57359,57359,57360,Kiganjo North Eastern,Kiambu,Thika,H-POLE STRUCTURE 50-315 KVA 11/0.433 K.V. (E /...,( CODE 2 ) - SUBSTATION,,THIKA,NDARUGU 11KV,,,,WITEITHIE,144081,283228.640235,9.881149e+06


**Observation**  

The transformers dataset is uniform from start to finish.

In [65]:
# Get metadata
transformers_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 57360 entries, 0 to 57359
Data columns (total 17 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   FID         57360 non-null  int64  
 1   OBJECTID    57360 non-null  int64  
 2   RCC1        57360 non-null  object 
 3   County2     57360 non-null  object 
 4   Branch3     57360 non-null  object 
 5   Substati9   57360 non-null  object 
 6   DCS_Cate15  57360 non-null  object 
 7   Modifyin25  57360 non-null  object 
 8   Origin_o27  57360 non-null  object 
 9   Feeder_o28  57360 non-null  object 
 10  Substati29  57360 non-null  object 
 11  HT_Isola32  57360 non-null  object 
 12  Road_Str38  57337 non-null  object 
 13  Physical39  57346 non-null  object 
 14  Road        57357 non-null  object 
 15  longitude   57360 non-null  float64
 16  latitude    57360 non-null  float64
dtypes: float64(2), int64(2), object(13)
memory usage: 7.4+ MB


**Observation**  

The distribution transformers dataset has 4 numerical columns and 13 categorical columns.


In [66]:
# statistical summary of numerical columns
transformers_df.describe().T

,count,mean,std,min,25%,50%,75%,max
FID,57360.0,2.867950e+04,16558.550057,0.000000e+00,1.433975e+04,2.867950e+04,4.301925e+04,5.735900e+04
OBJECTID,57360.0,2.868050e+04,16558.550057,1.000000e+00,1.434075e+04,2.868050e+04,4.302025e+04,5.736000e+04
longitude,57360.0,2.190842e+05,159114.410775,-6.230872e+04,7.656789e+04,2.502806e+05,2.952160e+05,8.186889e+05
latitude,57360.0,9.915628e+06,131054.226530,9.484751e+06,9.859711e+06,9.925214e+06,9.998688e+06,1.046754e+07


In [67]:
# statistical summary of categorical columns
transformers_df.describe(include='object').T

,count,unique,top,freq
RCC1,57360,10,Kiganjo Mt kenya,9141
County2,57360,49,Kiambu,4853
Branch3,57360,95,Eldoret Town,2189
Substati9,57360,22,UNKNOWN,33698
DCS_Cate15,57360,11,UNKNOWN,44513
Modifyin25,57360,2661,1020_POT_INS,38488
Origin_o27,57360,246,,5305
Feeder_o28,57360,846,MOI BARRACKS KITALE 33KV,1034
Substati29,57360,3,,42721
HT_Isola32,57360,7,TAPLINS,25282


## 3.12 KEN PowerPlants Data Understanding

### Power Plants — Column Dictionary

| Column    | Description |
|-----------|-------------|
| **PLANT** | Power plant name (site/facility identifier). |
| **STATUS**| Operational status (e.g., Operating, Under construction, Retired). |
| **SUM_MW** | Installed/available capacity in **megawatts (MW)**. |
| **COUNTRY** | Country where the plant is located (e.g., Kenya). |
| **LAT**   | Latitude (WGS84) of the plant. |
| **LON**   | Longitude (WGS84) of the plant. |
| **GEN_TYPE** | Primary generation type (e.g., Hydro, Geothermal, Thermal, Solar, Wind). |
| **ISO**   | Grid/market code or ISO-like region tag (if provided in source). |


In [68]:
#load datasets
powerplants_df = pd.read_csv("../Data/Power Grid and Infrastructure/KEN_PowerPlants.csv")

In [69]:
#preview first 5 rows
powerplants_df.head()

,PLANT,STATUS,SUM_MW,COUNTRY,LAT,LON,GEN_TYPE,ISO
0,ELDORET,OTHER,55.0,KENYA,0.516667,35.283333,THERMAL,KEN
1,EMBAKASI SUBSTATION,PLN,80.0,KENYA,-1.283333,36.816667,THERMAL,KEN
2,EWASO NGIRO,PLN,180.0,KENYA,0.566667,36.866667,HYDRO,KEN
3,GITARU,OPR,228.7,KENYA,-0.641700,37.675000,HYDRO,KEN
4,KAMBURU,OPR,94.2,KENYA,-0.850000,37.625000,HYDRO,KEN


In [70]:
# check the shape
print(f'The dataset has {powerplants_df.shape[0]} rows and {powerplants_df.shape[1]} columns')

The dataset has 32 rows and 8 columns


In [71]:
# checking the info of the dataset
powerplants_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32 entries, 0 to 31
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   PLANT     32 non-null     object 
 1   STATUS    32 non-null     object 
 2   SUM_MW    32 non-null     float64
 3   COUNTRY   32 non-null     object 
 4   LAT       32 non-null     float64
 5   LON       32 non-null     float64
 6   GEN_TYPE  32 non-null     object 
 7   ISO       32 non-null     object 
dtypes: float64(3), object(5)
memory usage: 2.1+ KB


**Observations:** The dataset has 4 categorical columns and 4 numerical columns with no null values.

In [72]:
#checking null values
powerplants_df.isnull().sum()

PLANT       0
STATUS      0
SUM_MW      0
COUNTRY     0
LAT         0
LON         0
GEN_TYPE    0
ISO         0
dtype: int64

In [73]:
#checking for duplicates
powerplants_df.duplicated().sum()

0

In [74]:
#check statistical summary of numerical columns
powerplants_df.describe().T

,count,mean,std,min,25%,50%,75%,max
SUM_MW,32.0,60.087187,48.521474,10.000000,30.000000,46.750000,70.997500,228.700000
LAT,32.0,-1.333802,1.731227,-4.050000,-1.975000,-0.842280,-0.564608,1.916700
LON,32.0,37.242419,1.677639,34.489167,36.350118,36.841667,38.337467,39.666667


In [75]:
#check statistical summary of categorical columns
powerplants_df.describe(include="O").T

,count,unique,top,freq
PLANT,32,26,KIPEVU,6
STATUS,32,4,OPR,18
COUNTRY,32,1,KENYA,32
GEN_TYPE,32,3,THERMAL,19
ISO,32,1,KEN,32


## 3.13 Network Data Understanding

### Transmission Lines — Column Dictionary

| Column        | Description |
|---------------|-------------|
| **COUNTRY**   | Country code/name for the line record (e.g., Kenya). |
| **CNTRY_NAME**| Full country name (redundant with `COUNTRY`; pick one). |
| **VOLTAGE_KV**| Nominal transmission voltage in **kilovolts** (e.g., 66, 132, 220, 400). |
| **FROM_NM**   | Name of the **origin** substation/bus for the line segment. |
| **TO_NM**     | Name of the **destination** substation/bus for the line segment. |
| **STATUS**    | Operational status of the line (e.g., In service, Under construction, Planned). |
| **SOURCES**   | Data/source reference or citation for this record. |
| **PROJECT_NM**| Project/program name associated with the line (often missing; present for a few records). |

In [76]:
#load dataset
network_df= pd.read_csv("../Data/Power Grid and Infrastructure/Kenya_Electricity_Transmission_Network_attributes.csv")

In [77]:
#preview first 5 rows
network_df.head()

,COUNTRY,CNTRY_NAME,VOLTAGE_KV,FROM_NM,TO_NM,STATUS,SOURCES,PROJECT_NM
0,KEN,KENYA,330,Arusha,Nairobi,Under Study,NaN,Zambia-Tanzania-Kenya Interconnector
1,KEN,KENYA,132,TORORO,ELDORET,Existing,UGANDA-AERDP Final Report Annex 1; February 2004,NaN
2,KEN,KENYA,220,TORORO,ELDORET,Planned,UGANDA-AERDP Final Report Annex 1; February 2004,NaN
3,KEN,KENYA,220,ELDORET,TURKWEL,Existing,WB map archive IBRD #33090; May 2004,NaN
4,KEN,KENYA,33,ELDORET,KIBIGORI,Existing,WB map archive IBRD #33090; May 2004,NaN


In [78]:
# preview the last 5 rows
network_df.tail()

,COUNTRY,CNTRY_NAME,VOLTAGE_KV,FROM_NM,TO_NM,STATUS,SOURCES,PROJECT_NM
43,KEN,KENYA,33,KIBIGORI,KERICHO,Existing,WB map archive IBRD #33090; May 2004,NaN
44,KEN,KENYA,33,KIBIGORI,KISUMU,Existing,WB map archive IBRD #33090; May 2004,NaN
45,KEN,KENYA,220,NAIROBI,MOMBASA,Planned,WB map archive IBRD #33090; May 2004,NaN
46,KEN,KENYA,330,Longonot,Ethiopian border,Under Study,Wb project document: P108921,Kenya-Ethiopia Interconnector
47,KEN,KENYA,220,Tanzania border,Nairobi,Under Study,WB map archive IBRD #35558; Sep 2007,East Africa Power Market Ugand-Tanzania-Kenya ...


**Observation**: The dataset is uniform from top to bottom

In [79]:
#check the shape
print(f'The dataset has {network_df.shape[0]} rows and {network_df.shape[1]} columns')

The dataset has 48 rows and 8 columns


In [80]:
# check the data information
network_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48 entries, 0 to 47
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   COUNTRY     48 non-null     object
 1   CNTRY_NAME  48 non-null     object
 2   VOLTAGE_KV  48 non-null     int64 
 3   FROM_NM     48 non-null     object
 4   TO_NM       48 non-null     object
 5   STATUS      48 non-null     object
 6   SOURCES     47 non-null     object
 7   PROJECT_NM  3 non-null      object
dtypes: int64(1), object(7)
memory usage: 3.1+ KB


**Observation**: The dataset has one numerical column and 6 categorical columns

In [81]:
#check statistical summary of numerical columns
network_df.describe().T

,count,mean,std,min,25%,50%,75%,max
VOLTAGE_KV,48.0,119.166667,78.746654,33.0,33.0,132.0,132.0,330.0


In [82]:
#check statistical summary of categorical columns
network_df.describe(include="O").T

,count,unique,top,freq
COUNTRY,48,1,KEN,48
CNTRY_NAME,48,1,KENYA,48
FROM_NM,48,23,NAIROBI,11
TO_NM,48,36,KINDARUMA,4
STATUS,48,3,Existing,43
SOURCES,47,4,WB map archive IBRD #33090; May 2004,43
PROJECT_NM,3,3,Zambia-Tanzania-Kenya Interconnector,1


## 2.14 Additional Weather Data Understanding

In [83]:
#import datasets
# preview the first few rows of the dataset
new_weather_df = pd.read_csv('../Data/Weather/era5_cleaned.csv')
new_weather_df.head()

,COUNTY,time,cape_mean_6h_mean,cape_mean_12h_mean,cape_mean_24h_mean,wind_mean_6h_mean,wind_mean_12h_mean,wind_mean_24h_mean,wind_max_6h_max,wind_max_12h_max,...,cloud_mean_24h_mean,total_precipitation_mm_mean,total_precipitation_mm_max,total_precipitation_mm_sum,hour_first,day_of_week_first,month_first,is_weekend_first,is_holiday_first,season_first
0,Baringo,2024-01-01 00:00:00,149.416667,149.416667,149.416667,0.809555,0.809555,0.809555,1.235864,1.235864,...,0.911971,0.027784,0.121117,0.416756,0,0,1,0,1,dry_cool
1,Baringo,2024-01-01 01:00:00,154.679167,154.679167,154.679167,0.825501,0.825501,0.825501,1.235864,1.235864,...,0.928203,0.047557,0.180721,0.713348,1,0,1,0,1,dry_cool
2,Baringo,2024-01-01 02:00:00,152.552778,152.552778,152.552778,0.844295,0.844295,0.844295,1.324105,1.324105,...,0.897692,0.067329,0.309467,1.009941,2,0,1,0,1,dry_cool
3,Baringo,2024-01-01 03:00:00,146.929167,146.929167,146.929167,0.844887,0.844887,0.844887,1.394833,1.394833,...,0.831795,0.049814,0.274658,0.747204,3,0,1,0,1,dry_cool
4,Baringo,2024-01-01 04:00:00,139.790000,139.790000,139.790000,0.844379,0.844379,0.844379,1.414635,1.414635,...,0.788850,0.002861,0.015259,0.042915,4,0,1,0,1,dry_cool


In [84]:
# check the tail to confirm uniformity
new_weather_df.tail()

,COUNTY,time,cape_mean_6h_mean,cape_mean_12h_mean,cape_mean_24h_mean,wind_mean_6h_mean,wind_mean_12h_mean,wind_mean_24h_mean,wind_max_6h_max,wind_max_12h_max,...,cloud_mean_24h_mean,total_precipitation_mm_mean,total_precipitation_mm_max,total_precipitation_mm_sum,hour_first,day_of_week_first,month_first,is_weekend_first,is_holiday_first,season_first
726105,West Pokot,2025-10-19 12:00:00,198.518092,153.069627,243.722588,2.169606,1.766332,1.512562,3.716106,3.716106,...,0.606445,0.914059,16.573906,34.734250,12,6,10,1,0,short_rains
726106,West Pokot,2025-10-19 13:00:00,208.469846,152.179002,239.133909,1.974275,1.808966,1.484187,3.634701,3.716106,...,0.603222,1.944416,32.192230,73.887825,13,6,10,1,0,short_rains
726107,West Pokot,2025-10-19 14:00:00,220.718202,158.334978,232.945175,1.779532,1.847872,1.471427,3.391531,3.716106,...,0.603778,3.162208,53.350450,120.163920,14,6,10,1,0,short_rains
726108,West Pokot,2025-10-19 15:00:00,228.638706,175.694627,224.573328,1.650664,1.883596,1.472720,3.790390,3.790390,...,0.601213,3.842906,52.215576,146.030430,15,6,10,1,0,short_rains
726109,West Pokot,2025-10-19 16:00:00,219.616776,197.544682,218.444627,1.593170,1.889395,1.487167,3.790390,3.790390,...,0.603367,2.970294,28.017044,112.871170,16,6,10,1,0,short_rains


Observations:
- The dataset is uniform at the top and bottom

In [85]:
# checking the info of the dataset
new_weather_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 726110 entries, 0 to 726109
Data columns (total 23 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   COUNTY                       726110 non-null  object 
 1   time                         726110 non-null  object 
 2   cape_mean_6h_mean            726110 non-null  float64
 3   cape_mean_12h_mean           726110 non-null  float64
 4   cape_mean_24h_mean           726110 non-null  float64
 5   wind_mean_6h_mean            726110 non-null  float64
 6   wind_mean_12h_mean           726110 non-null  float64
 7   wind_mean_24h_mean           726110 non-null  float64
 8   wind_max_6h_max              726110 non-null  float64
 9   wind_max_12h_max             726110 non-null  float64
 10  wind_max_24h_max             726110 non-null  float64
 11  cloud_mean_6h_mean           726110 non-null  float64
 12  cloud_mean_12h_mean          726110 non-null  float64
 13 

**Observations:**
- The dataset has 20 numerical columns and 3 categorical columns
- We will change time to datetime

In [86]:
# checking column names
new_weather_df.columns

Index(['COUNTY', 'time', 'cape_mean_6h_mean', 'cape_mean_12h_mean',
       'cape_mean_24h_mean', 'wind_mean_6h_mean', 'wind_mean_12h_mean',
       'wind_mean_24h_mean', 'wind_max_6h_max', 'wind_max_12h_max',
       'wind_max_24h_max', 'cloud_mean_6h_mean', 'cloud_mean_12h_mean',
       'cloud_mean_24h_mean', 'total_precipitation_mm_mean',
       'total_precipitation_mm_max', 'total_precipitation_mm_sum',
       'hour_first', 'day_of_week_first', 'month_first', 'is_weekend_first',
       'is_holiday_first', 'season_first'],
      dtype='object')

**Observations:**
- Need to standardize column names to lowercase

In [87]:
#data summary statistics
new_weather_df.describe(include='O').T

,count,unique,top,freq
COUNTY,726110,46,Baringo,15785
time,726110,15785,2024-01-01 00:00:00,46
season_first,726110,4,dry_warm,269376


In [88]:
#data summary statistics
new_weather_df.describe().T

,count,mean,std,min,25%,50%,75%,max
cape_mean_6h_mean,726110.0,154.479794,256.851485,0.000000,2.812500,35.812500,194.828369,3126.087500
cape_mean_12h_mean,726110.0,154.483864,242.748788,0.000000,4.377604,45.744699,203.936701,2990.185985
cape_mean_24h_mean,726110.0,154.570344,228.921843,0.000000,6.339844,58.085570,209.869792,2764.762311
wind_mean_6h_mean,726110.0,2.515202,1.589028,0.115798,1.369375,2.047503,3.187148,13.312162
wind_mean_12h_mean,726110.0,2.515148,1.536204,0.115798,1.423525,2.046312,3.117451,12.956526
wind_mean_24h_mean,726110.0,2.514830,1.487281,0.115798,1.457700,2.052081,3.053415,12.752321
wind_max_6h_max,726110.0,4.269254,2.294429,0.115798,2.434201,3.752623,5.712128,16.547562
wind_max_12h_max,726110.0,4.682901,2.289521,0.115798,2.880072,4.165942,6.147765,16.547562
wind_max_24h_max,726110.0,5.091619,2.271813,0.115798,3.300108,4.559870,6.555263,16.547562
cloud_mean_6h_mean,726110.0,0.607191,0.290655,0.000000,0.363877,0.638467,0.879536,1.000000


# Chapter 3: Data Cleaning

## 3.1 Outages Data Cleaning

In [89]:
# check and drop null values
print(outage_df.isna().sum())

outage_df.drop(columns=['Date','source_file','Unnamed: 9', 'Unnamed: 10'], inplace=True)

# reality check
outage_df.isna().sum()

region                   0
county                   0
area                     0
date                     0
Date                   550
start_time               0
end_time                 0
affected_locations       0
source_file           1453
Unnamed: 9            2301
Unnamed: 10           2302
dtype: int64


region                0
county                0
area                  0
date                  0
start_time            0
end_time              0
affected_locations    0
dtype: int64

In [90]:
# check and drop duplicates
outage_df.duplicated().sum()
outage_df.drop_duplicates(inplace=True)

# reality check
print(f' We have {outage_df.duplicated().sum()} duplicate rows')

 We have 0 duplicate rows


In [91]:
# Checking shape of the dataset
print(f'There are {outage_df.shape[0]} entries and {outage_df.shape[1]} columns')

There are 2255 entries and 7 columns


In [92]:
outage_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2255 entries, 0 to 2302
Data columns (total 7 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   region              2255 non-null   object
 1   county              2255 non-null   object
 2   area                2255 non-null   object
 3   date                2255 non-null   object
 4   start_time          2255 non-null   object
 5   end_time            2255 non-null   object
 6   affected_locations  2255 non-null   object
dtypes: object(7)
memory usage: 140.9+ KB


In [93]:
#unique values in each column
for col in outage_df.columns:
    print(f'{col}: \n{outage_df[col].unique()} unique values\n')

region: 
['NORTH RIFT' 'WESTERN' 'NORTH EASTERN' 'NAIROBI' 'CENTRAL RIFT'
 'MT. KENYA' 'SOUTH NYANZA' 'MT KENYA' 'NORTH RIFT ' 'NORTH EASTERN '
 'NAIROBI ' 'WESTERN ' 'SOUTH NYANZA ' 'MT. KENYA ' 'COAST '
 'CENTRAL RIFT ' 'MT.KENYA' 'COAST' 'CENTAL RIFT' 'NAIROOBI'] unique values

county: 
['UASIN GISHU' 'TRANS NZOIA' 'BUSIA' 'KIAMBU' 'NAIROBI' 'KAJIADO' 'BOMET'
 'KISUMU' 'NYERI' 'THARAKA NITHI' 'MAKUENI' 'KIRINYAGA' 'MIGORI' 'HOMABAY'
 'NAKURU' 'KISII' "MURANG'A" 'KAKAMEGA' 'NYAMIRA' 'EMBU' 'ELGEYO MARAKWET'
 'MERU' 'BUNGOMA' 'VIHIGA' 'NYANDARUA' 'LAIKIPIA' 'GARISSA' 'UASIN GISHU '
 'KIAMBU ' 'NAIROBI ' 'MACHAKOS ' 'NANDI ' 'KISUMU ' 'MIGORI ' 'MERU '
 'KISII ' 'LAIKIPIA ' "MURANG'A " 'KITUI ' 'MOMBASA ' 'KWALE '
 'TANA RIVER ' 'KAKAMEGA ' 'NYERI ' 'EMBU ' 'KAJIADO ' 'ISIOLO '
 'BUNGOMA ' 'NAKURU ' 'TRANS NZOIA ' 'KIRINYAGA ' 'NYAMIRA ' 'MAKUENI '
 'HOMA BAY ' 'KILIFI ' 'KERICHO' 'WEST POKOT' 'BUSIA ' 'SIAYA ' 'NAROK'
 'HOMABAY ' 'BOMET ' 'THARAKA NITHI ' 'TURKANA ' 'USAIN GISHU' 'MOM

observations:
- Ensure all column values are standardized
- 

In [94]:
# standardize region column
# Strip spaces and normalize case
outage_df['region'] = outage_df['region'].str.strip().str.upper()

# Define merge
region_map = {
    'NORTH RIFT ': 'NORTH RIFT',
    'NORTH RIFT': 'NORTH RIFT',
    'WESTERN ': 'WESTERN',
    'WESTERN': 'WESTERN',
    'NORTH EASTERN ': 'NORTH EASTERN',
    'NORTH EASTERN': 'NORTH EASTERN',
    'NAIROBI ': 'NAIROBI',
    'NAIROBI': 'NAIROBI',
    'NAIROOBI': 'NAIROBI',
    'CENTRAL RIFT ': 'CENTRAL RIFT',
    'CENTRAL RIFT': 'CENTRAL RIFT',
    'CENTAL RIFT': 'CENTRAL RIFT',
    'COAST ': 'COAST',
    'COAST': 'COAST',
    'MT: KENYA': 'MT KENYA',
    'MT KENYA': 'MT KENYA',
    'MT:KENYA': 'MT KENYA',
    'MT: KENYA ': 'MT KENYA',
    'SOUTH NYANZA': 'SOUTH NYANZA',
    'SOUTH NYANZA ': 'SOUTH NYANZA',
    'SOUTH NYANZA & CENTRAL RIFT S': 'SOUTH NYANZA',  
}

# Apply the mapping
outage_df['region'] = outage_df['region'].replace(region_map)

# Verify cleanup
print(sorted(outage_df['region'].unique()))


['CENTRAL RIFT', 'COAST', 'MT KENYA', 'MT. KENYA', 'MT.KENYA', 'NAIROBI', 'NORTH EASTERN', 'NORTH RIFT', 'SOUTH NYANZA', 'WESTERN']


In [95]:
# standardize county column
# Strip spaces and normalize case
outage_df['county'] = outage_df['county'].str.strip().str.upper()
county_map = {
    'UASIN GISHU ': 'UASIN GISHU', 'USAIN GISHU': 'UASIN GISHU',
    'NAIROOBI': 'NAIROBI', 'NAIROBI ': 'NAIROBI',
    'KIAMBU ': 'KIAMBU', "MURANG'A ": "MURANG'A", 'MURANGÄ': "MURANG'A",
    'MURANGA': "MURANG'A",
    'MACHAKOS ': 'MACHAKOS', 'MACHACKOS': 'MACHAKOS', 'MACHKOS': 'MACHAKOS',
    'KITUI ': 'KITUI', 'ISIOLO ': 'ISIOLO', 'MERU ': 'MERU',
    'MOMBASA ': 'MOMBASA', ' MOMBASA': 'MOMBASA',
    'KWALE ': 'KWALE', 'KILIFI ': 'KILIFI', 'TANA RIVER ': 'TANA RIVER',
    'HOMA BAY ': 'HOMABAY', 'HOMABAY ': 'HOMABAY',
    'ELGEYO MARAKWET ': 'ELGEYO MARAKWET',
    'BARINGO ': 'BARINGO', 'GARISSA ': 'GARISSA',
    'NYANDARUA ': 'NYANDARUA', 'VIHIGA ': 'VIHIGA',
    'NYERI ': 'NYERI', 'NYAMIRA ': 'NYAMIRA', 'KAKAMEGA ': 'KAKAMEGA',
    'KAKAMENGA': 'KAKAMEGA', 'KIRINYAGA ': 'KIRINYAGA',
    "KIRINYANG'A ": 'KIRINYAGA', "KIRINYANG'A": 'KIRINYAGA',
    'KIRINYANGA': 'KIRINYAGA','HOMA BAY': 'HOMABAY',
    'TRANS NZOIA ': 'TRANS NZOIA', 'BOMET ': 'BOMET',
    'THARAKA NITHI ': 'THARAKA NITHI',
    'CENTRAL RIFT ': 'CENTRAL RIFT',
    'WEST POKOT ': 'WEST POKOT', 'SAMBURU ': 'SAMBURU',
    'TAITA TAVETA ': 'TAITA TAVETA',
    'NAROK ': 'NAROK', 'NANDI ': 'NANDI',
    'NAKURU  & KIAMBU ': 'NAKURU & KIAMBU',
    'NYAMIRA & KISII': 'KISII & NYAMIRA',  
    "MURANG'A/KIAMBU": "MURANG'A & KIAMBU",
    'KERICHO, BOMET, NYAMIRA, KISII, MIGORI & HOMABAY COUNTIES': 
        'SOUTH NYANZA REGION GROUP',
}
outage_df['county'] = outage_df['county'].replace(county_map)

#sanity check # Verify cleanup
print(sorted(outage_df['county'].unique()))


['BARINGO', 'BOMET', 'BUNGOMA', 'BUSIA', 'ELGEYO MARAKWET', 'EMBU', 'GARISSA', 'HOMABAY', 'ISIOLO', 'KAJIADO', 'KAKAMEGA', 'KERICHO', 'KIAMBU', 'KILIFI', 'KIRINYAGA', 'KISII', 'KISUMU', 'KITUI', 'KWALE', 'LAIKIPIA', 'LAMU', 'MACHAKOS', 'MAKUENI', 'MARSABIT', 'MERU', 'MIGORI', 'MOMBASA', "MURANG'A", 'NAIROBI', 'NAKURU', 'NANDI', 'NAROK', 'NKUBU', 'NYAMIRA', 'NYANDARUA', 'NYERI', 'SAMBURU', 'SIAYA', 'TAITA TAVETA', 'TANA RIVER', 'THARAKA NITHI', 'TRANS NZOIA', 'TURKANA', 'UASIN GISHU', 'VIHIGA', 'WEST POKOT']


In [96]:
#convert date column to datetime.date format
outage_df['date'] = pd.to_datetime(outage_df['date'].astype(str).str.strip(), dayfirst=True, errors='coerce').dt.date


In [97]:
# convert start_time and end_time to datetime.time format
for col in ['start_time', 'end_time']:
    outage_df[col] = pd.to_datetime(
        outage_df[col],   
        errors='coerce'      
    ).dt.time

#sanity check
outage_df

,region,county,area,date,start_time,end_time,affected_locations
0,NORTH RIFT,UASIN GISHU,SINENDET CENTRE,2024-06-29,10:00:00,17:00:00,"Sinendet Centre, Sakam Quarry, Lower Kipkaren ..."
1,NORTH RIFT,TRANS NZOIA,"KITALE TOWN, MARAMBACHI",2024-06-29,09:00:00,12:00:00,"Wamuini Centre, St. Joseph's Boys High Sch, St..."
2,WESTERN,BUSIA,"TANGA, KONA, NAMBALE",2024-06-29,09:00:00,17:00:00,"Nasira, Kajoro, Kololo Farm, Nambale DC, Jua K..."
3,NORTH EASTERN,KIAMBU,"KIKUYU TOWN, SIGONA",2024-06-29,12:00:00,16:00:00,"Kikuyu Town, Kidfarmaco, Gitaru, Regen, Magana..."
4,NAIROBI,NAIROBI,PART OF EASTLEIGH,2024-06-30,09:00:00,17:00:00,"Mohammed Yusuf Haji Ave, Ismail Hassan, Moyale..."
...,...,...,...,...,...,...,...
2298,NORTH EASTERN,KIAMBU,"KIGWARU, RUAKA, MUCHATHA",2025-10-26,09:00:00,16:00:00,"Little Sisters, Guango, Clifftop, Kigwaru-Inn,..."
2299,NORTH EASTERN,KIAMBU,JUJA INDUSTRIAL PARK,2025-10-26,09:00:00,16:00:00,"Juja International Abattoir Limited, Juja Deve..."
2300,NORTH EASTERN,KIAMBU,KIHINGO,2025-10-29,09:00:00,17:00:00,"Kihingo ACK, Kihingo 1st, 2nd, & 3rd Aves & ad..."
2301,NORTH EASTERN,KIAMBU,"NAZARETH, KIMORORI",2025-10-30,09:00:00,17:00:00,"Gatatha Farmers, Kimorori, Kawaida, Black Peta..."


observations:
- Date and time are now in datetime format
- we check summary to see if we have changes

In [98]:
outage_df.isna().sum()

region                0
county                0
area                  0
date                  0
start_time            0
end_time              0
affected_locations    0
dtype: int64

In [99]:
#data summary statistics
outage_df.describe(include='all').T

,count,unique,top,freq
region,2255,10,NAIROBI,451
county,2255,46,NAIROBI,342
area,2255,1820,PART OF PARKLANDS,14
date,2255,357,2025-04-03,23
start_time,2255,14,09:00:00,1701
end_time,2255,16,17:00:00,1879
affected_locations,2255,2094,"Kipkenyo, Kosyin, Leseru, Seiyot & adjacent cu...",5


observations:
- Mt Kenya Region and Nairobi county has the highest outages
- Parts of Parklands is the most affected area in terms of outages
- On 3/04/2025, we had the most outages
    - Planned maintenance affected many feeders
    - This could be as a result of heavy rains in March
- We have most outages happening between 9AM - 5PM

In [100]:
# save cleaned outage data
outage_df.to_csv('../cleaned_datasets/cleaned_outages.csv',index=False)

In [101]:
# Combine date and time into single datetime columns 
# Convert date/time columns to strings before combining
outage_df['start_datetime'] = pd.to_datetime(
    outage_df['date'].astype(str) + ' ' + outage_df['start_time'].astype(str),
    errors='coerce'
)

outage_df['end_datetime'] = pd.to_datetime(
    outage_df['date'].astype(str) + ' ' + outage_df['end_time'].astype(str),
    errors='coerce'
)

# Handle overnight outages 
# If the end time is earlier than the start time, assume it passed midnight → add one day
outage_df.loc[outage_df['end_datetime'] < outage_df['start_datetime'], 'end_datetime'] += pd.Timedelta(days=1)


# Expand each outage event into hourly records 
def expand_to_hours(row):
    
    # If start or end datetime is missing, skip
    if pd.isna(row['start_datetime']) or pd.isna(row['end_datetime']):
        return pd.DataFrame()

    # Create hourly timestamps between start and end (rounded)
    hours = pd.date_range(
        start=row['start_datetime'].floor('H'),
        end=row['end_datetime'].ceil('H'),
        freq='H'
    )

    # Create one row per hour
    return pd.DataFrame({
        'region': row.get('region', np.nan),
        'county': row.get('county', np.nan),
        'area': row.get('area', np.nan),
        'hour': hours,
        'affected_locations': row.get('affected_locations', 0)
    })


# Apply the expansion to all rows 
expanded_list = outage_df.apply(expand_to_hours, axis=1)

# Combine all hourly records into a single DataFrame
hourly_outage_df = pd.concat(expanded_list.tolist(), ignore_index=True)

# Aggregate overlapping outages (same area/hour) 
hourly_summary = (
    hourly_outage_df
    .groupby(['region', 'county', 'area', 'hour'], as_index=False)
    .agg({'affected_locations': 'sum'})
)

# Add time-based features for modeling 
hourly_summary['hour_of_day'] = hourly_summary['hour'].dt.hour
hourly_summary['day_of_week'] = hourly_summary['hour'].dt.day_name()
hourly_summary['month'] = hourly_summary['hour'].dt.month
hourly_summary['year'] = hourly_summary['hour'].dt.year
hourly_summary['is_weekend'] = hourly_summary['day_of_week'].isin([5,6]).astype(int)
hourly_summary['is_night'] = hourly_summary['hour_of_day'].between(0, 6).astype(int)

#sort for readability
hourly_summary = hourly_summary.sort_values(['region', 'county', 'area', 'hour']).reset_index(drop=True)

# Preview
hourly_summary.head()

,region,county,area,hour,affected_locations,hour_of_day,day_of_week,month,year,is_weekend,is_night
0,CENTRAL RIFT,BARINGO,MOGOTIO,2025-02-20 09:00:00,"Mogotio Sub-County Offices,RavineJunction,Kamn...",9,Thursday,2,2025,0,0
1,CENTRAL RIFT,BARINGO,MOGOTIO,2025-02-20 10:00:00,"Mogotio Sub-County Offices,RavineJunction,Kamn...",10,Thursday,2,2025,0,0
2,CENTRAL RIFT,BARINGO,MOGOTIO,2025-02-20 11:00:00,"Mogotio Sub-County Offices,RavineJunction,Kamn...",11,Thursday,2,2025,0,0
3,CENTRAL RIFT,BARINGO,MOGOTIO,2025-02-20 12:00:00,"Mogotio Sub-County Offices,RavineJunction,Kamn...",12,Thursday,2,2025,0,0
4,CENTRAL RIFT,BARINGO,MOGOTIO,2025-02-20 13:00:00,"Mogotio Sub-County Offices,RavineJunction,Kamn...",13,Thursday,2,2025,0,0


In [102]:
hourly_summary['outage_count'] = 1  
# hourly_summary = (
#     hourly_summary.groupby(['region', 'county', 'area', 'hour'], as_index=False)
#     .agg({'outage_count': 'sum', 'affected_locations': 'sum'})


hourly_summary = hourly_summary.sort_values(['region', 'hour'])
hourly_summary['outage_lag_1'] = hourly_summary.groupby('region')['outage_count'].shift(1)
hourly_summary['outage_rolling_6h'] = hourly_summary.groupby('region')['outage_count'].rolling(6, min_periods=1).mean().reset_index(0, drop=True)
hourly_summary['outage_rolling_24h'] = hourly_summary.groupby('region')['outage_count'].rolling(24, min_periods=1).mean().reset_index(0, drop=True)
hourly_summary.head()

,region,county,area,hour,affected_locations,hour_of_day,day_of_week,month,year,is_weekend,is_night,outage_count,outage_lag_1,outage_rolling_6h,outage_rolling_24h
59,CENTRAL RIFT,BOMET,"KAPKOROS, KIPLOKYI",2024-07-01 09:00:00,"Kapkoros Mkt, Kapkoros KTDA T/Fact, Ndaraweta ...",9,Monday,7,2024,0,0,1,NaN,1.0,1.0
60,CENTRAL RIFT,BOMET,"KAPKOROS, KIPLOKYI",2024-07-01 10:00:00,"Kapkoros Mkt, Kapkoros KTDA T/Fact, Ndaraweta ...",10,Monday,7,2024,0,0,1,1.0,1.0,1.0
61,CENTRAL RIFT,BOMET,"KAPKOROS, KIPLOKYI",2024-07-01 11:00:00,"Kapkoros Mkt, Kapkoros KTDA T/Fact, Ndaraweta ...",11,Monday,7,2024,0,0,1,1.0,1.0,1.0
62,CENTRAL RIFT,BOMET,"KAPKOROS, KIPLOKYI",2024-07-01 12:00:00,"Kapkoros Mkt, Kapkoros KTDA T/Fact, Ndaraweta ...",12,Monday,7,2024,0,0,1,1.0,1.0,1.0
63,CENTRAL RIFT,BOMET,"KAPKOROS, KIPLOKYI",2024-07-01 13:00:00,"Kapkoros Mkt, Kapkoros KTDA T/Fact, Ndaraweta ...",13,Monday,7,2024,0,0,1,1.0,1.0,1.0


## 3.2 Kenya Weather Data Cleaning

In [103]:
#check shape of the dataset
print(f'There are {weather_df.shape[0]} entries and {weather_df.shape[1]} columns')

There are 31255 entries and 10 columns


In [104]:
for col in ['PRECTOTCORR', 'ALLSKY_SFC_SW_DWN', 'ALLSKY_KT']:
    weather_df[col] = weather_df[col].replace(-999, np.nan)
    weather_df[col].fillna(weather_df[col].mean(), inplace=True)


In [105]:
#checking for missing values
weather_df.isna().sum()

county               0
YEAR                 0
MO                   0
DY                   0
date                 0
PRECTOTCORR          0
ALLSKY_SFC_SW_DWN    0
ALLSKY_KT            0
latitude             0
longitude            0
dtype: int64

In [106]:
#checking for duplicates
print(f' There are {weather_df.duplicated().sum()} duplicates')

 There are 0 duplicates


In [107]:
#convert date column to datetime.date format
weather_df['date'] = pd.to_datetime(weather_df['date'].astype(str).str.strip(),dayfirst=True, errors='coerce').dt.date


In [108]:
weather_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31255 entries, 0 to 31254
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   county             31255 non-null  object 
 1   YEAR               31255 non-null  int64  
 2   MO                 31255 non-null  int64  
 3   DY                 31255 non-null  int64  
 4   date               31255 non-null  object 
 5   PRECTOTCORR        31255 non-null  float64
 6   ALLSKY_SFC_SW_DWN  31255 non-null  float64
 7   ALLSKY_KT          31255 non-null  float64
 8   latitude           31255 non-null  float64
 9   longitude          31255 non-null  float64
dtypes: float64(5), int64(3), object(2)
memory usage: 2.4+ MB


In [109]:
#sanity check
weather_df

,county,YEAR,MO,DY,date,PRECTOTCORR,ALLSKY_SFC_SW_DWN,ALLSKY_KT,latitude,longitude
0,Turkana,2024,1,1,2024-01-01,2.610000,5.229100,0.550000,3.412964,35.434957
1,Turkana,2024,1,2,2024-01-02,0.020000,5.724000,0.600000,3.412964,35.434957
2,Turkana,2024,1,3,2024-01-03,0.020000,5.761900,0.610000,3.412964,35.434957
3,Turkana,2024,1,4,2024-01-04,0.000000,6.375600,0.670000,3.412964,35.434957
4,Turkana,2024,1,5,2024-01-05,0.000000,5.997400,0.630000,3.412964,35.434957
...,...,...,...,...,...,...,...,...,...,...
31250,Mombasa,2025,10,22,2025-10-22,0.190000,5.705858,0.575026,-4.017651,39.654620
31251,Mombasa,2025,10,23,2025-10-23,0.670000,5.705858,0.575026,-4.017651,39.654620
31252,Mombasa,2025,10,24,2025-10-24,3.884708,5.705858,0.575026,-4.017651,39.654620
31253,Mombasa,2025,10,25,2025-10-25,3.884708,5.705858,0.575026,-4.017651,39.654620


In [110]:
#rename columns for easier access
weather_df.rename(columns={
    'PRECTOTCORR': 'rainfall_mm',
    'ALLSKY_SFC_SW_DWN': 'solar_energy_wm2',
    'ALLSKY_KT': 'sky_clarity_index'
}, inplace=True)

In [111]:
#check unique values
for col in weather_df.columns:
    print(f'{col}: \n{weather_df[col].unique()} unique values\n')

county: 
['Turkana' 'Marsabit' 'Mandera' 'Wajir' 'West Pokot' 'Samburu' 'Isiolo'
 'Baringo' 'Keiyo-Marakwet' 'Trans Nzoia' 'Bungoma' 'Garissa'
 'Uasin Gishu' 'Kakamega' 'Laikipia' 'Busia' 'Meru' 'Nandi' 'Siaya'
 'Nakuru' 'Vihiga' 'Nyandarua' 'Tharaka' 'Kericho' 'Kisumu' 'Nyeri'
 'Tana River' 'Kitui' 'Kirinyaga' 'Embu' 'Homa Bay' 'Bomet' 'Nyamira'
 'Narok' 'Kisii' "Murang'a" 'Migori' 'Kiambu' 'Machakos' 'Kajiado'
 'Nairobi' 'Makueni' 'Lamu' 'Kilifi' 'Taita Taveta' 'Kwale' 'Mombasa'] unique values

YEAR: 
[2024 2025] unique values

MO: 
[ 1  2  3  4  5  6  7  8  9 10 11 12] unique values

DY: 
[ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 29 30 31] unique values

date: 
[datetime.date(2024, 1, 1) datetime.date(2024, 1, 2)
 datetime.date(2024, 1, 3) datetime.date(2024, 1, 4)
 datetime.date(2024, 1, 5) datetime.date(2024, 1, 6)
 datetime.date(2024, 1, 7) datetime.date(2024, 1, 8)
 datetime.date(2024, 1, 9) datetime.date(2024, 1, 10)
 datetime.date(20

In [112]:
weather_df.describe().T

,count,mean,std,min,25%,50%,75%,max
YEAR,31255.0,2024.449624,0.497464,2024.000000,2024.000000,2024.000000,2025.000000,2025.000000
MO,31255.0,6.036090,3.232671,1.000000,3.000000,6.000000,9.000000,12.000000
DY,31255.0,15.637594,8.760239,1.000000,8.000000,16.000000,23.000000,31.000000
rainfall_mm,31255.0,3.884708,7.852904,0.000000,0.150000,0.990000,3.960000,267.050000
solar_energy_wm2,31255.0,5.705858,1.099408,0.986900,5.046350,5.835400,6.505100,8.176600
sky_clarity_index,31255.0,0.575026,0.092795,0.110000,0.530000,0.575026,0.640000,0.780000
latitude,31255.0,-0.336641,1.621684,-4.139058,-1.253919,-0.343076,0.532028,3.438400
longitude,31255.0,36.785633,1.883245,34.193840,35.112300,36.819685,37.871708,40.766286


observations:
- After filling null values in rainfall, solar energy and sky clarity, the values are now within range of what we experience in Kenya in a day
- We have the weather conditions from 2024 to 2025
- All values in the dataset are standardized

In [113]:
#save cleaned weather data
weather_df.to_csv('../cleaned_datasets/cleaned_Kenya_weather.csv',index=False)

## 3.3 Weather Hourly Data Cleaning

In [114]:
# Check nulls
weather_hourly_df.isna().sum()

admin_unit         0
timestamp_local    0
rain_6h            0
rain_24h           0
wind_max_24h       0
temp_max_24h       0
precipitation      0
wind_gusts_10m     0
temperature_2m     0
dtype: int64

In [115]:
# Check duplicates 
print(f' There are {weather_hourly_df.duplicated().sum()} duplicates')

 There are 0 duplicates


In [116]:
# Convert to datetime 
weather_hourly_df['timestamp_local'] = pd.to_datetime(weather_hourly_df['timestamp_local'], errors='coerce')

# Create separate date and time columns
weather_hourly_df['date'] = weather_hourly_df['timestamp_local'].dt.date
weather_hourly_df['time'] = weather_hourly_df['timestamp_local'].dt.time

#dropping the original timestamp_local column
weather_hourly_df.drop('timestamp_local', axis=1, inplace=True)
weather_hourly_df[:3]


,admin_unit,rain_6h,rain_24h,wind_max_24h,temp_max_24h,precipitation,wind_gusts_10m,temperature_2m,date,time
0,Mombasa,0.0,0.0,31.3,24.6,0.0,31.3,24.6,2024-09-16,00:00:00
1,Mombasa,0.0,0.0,31.3,24.6,0.0,30.2,24.5,2024-09-16,01:00:00
2,Mombasa,0.0,0.0,31.3,24.6,0.0,29.2,24.5,2024-09-16,02:00:00


In [117]:
#checking unique values
for col in weather_hourly_df.columns:
    print(f'{col}: \n{weather_hourly_df[col].unique()} unique values\n')

admin_unit: 
['Mombasa' 'Kwale' 'Kilifi' 'Tana River' 'Lamu' 'Taita-Taveta' 'Garissa'
 'Wajir' 'Mandera' 'Marsabit' 'Isiolo' 'Meru' 'Tharaka-Nithi' 'Embu'
 'Kitui' 'Machakos' 'Makueni' 'Nyandarua' 'Nyeri' 'Kirinyaga' "Murang'a"
 'Kiambu' 'Turkana' 'West Pokot' 'Samburu' 'Trans Nzoia' 'Uasin Gishu'
 'Elgeyo-Marakwet' 'Nandi' 'Baringo' 'Laikipia' 'Nakuru' 'Narok' 'Kajiado'
 'Kericho' 'Bomet' 'Kakamega' 'Vihiga' 'Bungoma' 'Busia' 'Siaya' 'Kisumu'
 'Homa Bay' 'Migori' 'Kisii' 'Nyamira' 'Nairobi'] unique values

rain_6h: 
[ 0.   0.1  0.2 ... 23.8 14.8 24.2] unique values

rain_24h: 
[0.  0.1 0.2 ... 9.9 0.9 0.4] unique values

wind_max_24h: 
[31.3 35.3 38.2 39.2 40.7 41.  43.6 45.7 46.1 45.  43.2 42.1 37.4 38.5
 40.3 37.1 41.4 44.6 45.4 46.4 40.  39.6 36.7 34.6 38.9 42.8 43.9 50.4
 51.8 53.6 56.2 57.6 57.2 51.1 47.9 37.8 41.8 47.2 49.  36.4 44.3 42.5
 36.  34.2 34.9 33.8 33.1 50.8 35.6 32.4 31.7 32.  27.7 29.2 30.6 32.8
 33.5 30.2 31.  29.9 23.  25.9 28.8 29.5 28.1 28.4 49.7 49.3 58.3 56.5


observations:
- change admin unit column to county for uniformity
- All unque variables are standardized

In [118]:
#renaming admin_unit to county for easier merging
weather_hourly_df.rename(columns={'admin_unit': 'county'}, inplace=True)
weather_hourly_df.columns

Index(['county', 'rain_6h', 'rain_24h', 'wind_max_24h', 'temp_max_24h',
       'precipitation', 'wind_gusts_10m', 'temperature_2m', 'date', 'time'],
      dtype='object')

In [119]:
# final sanity check
weather_hourly_df[:3]

,county,rain_6h,rain_24h,wind_max_24h,temp_max_24h,precipitation,wind_gusts_10m,temperature_2m,date,time
0,Mombasa,0.0,0.0,31.3,24.6,0.0,31.3,24.6,2024-09-16,00:00:00
1,Mombasa,0.0,0.0,31.3,24.6,0.0,30.2,24.5,2024-09-16,01:00:00
2,Mombasa,0.0,0.0,31.3,24.6,0.0,29.2,24.5,2024-09-16,02:00:00


In [120]:
# save cleaned hourly weather data
weather_hourly_df.to_csv('../cleaned_datasets/cleaned_weather_hourly.csv',index=False)

## 3.4 Vegetation data cleaning

In [121]:
# Check nulls
vegetation_df.isna().sum()

COUNTY       0
date         0
ndvi_mean    0
ndvi_std     0
ndvi_min     0
ndvi_max     0
dtype: int64

In [122]:
# Check duplicates
print(f' There are {vegetation_df.duplicated().sum()} duplicates')

 There are 0 duplicates


In [123]:
# standardize column names
vegetation_df.columns=vegetation_df.columns.str.lower()
vegetation_df.columns

Index(['county', 'date', 'ndvi_mean', 'ndvi_std', 'ndvi_min', 'ndvi_max'], dtype='object')

In [124]:
# save cleaned vegetaion data
vegetation_df.to_csv('../cleaned_datasets/cleaned_vegetation_data.csv')

## 3.5 Population Density Data Cleaning

In [125]:
# Check nulls
population_df.isna().sum()

county_code                0
county                     0
population                 0
male                       0
female                     0
intersex                   0
sex_ratio                  0
area_km2                   0
density_per_km2            0
number_of_households       0
avg_household_size         0
percent_of_national_pop    0
dtype: int64

In [126]:
# Check duplicates
print(f'There are {population_df.duplicated().sum()} duplicates')

There are 0 duplicates


In [127]:
#check unique values
for col in population_df.columns:
    print(f'{col}: \n{population_df[col].unique()} unique values\n')

county_code: 
[ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47] unique values

county: 
['Mombasa' 'Kwale' 'Kilifi' 'Tana River' 'Lamu' 'Taita Taveta' 'Garissa'
 'Wajir' 'Mandera' 'Marsabit' 'Isiolo' 'Meru' 'Tharaka Nithi' 'Embu'
 'Kitui' 'Machakos' 'Makueni' 'Nyandarua' 'Nyeri' 'Kirinyaga' "Murang'a"
 'Kiambu' 'Turkana' 'West Pokot' 'Samburu' 'Trans Nzoia' 'Uasin Gishu'
 'Elgeyo Marakwet' 'Nandi' 'Baringo' 'Laikipia' 'Nakuru' 'Narok' 'Kajiado'
 'Kericho' 'Bomet' 'Kakamega' 'Vihiga' 'Bungoma' 'Busia' 'Siaya' 'Kisumu'
 'Homa Bay' 'Migori' 'Kisii' 'Nyamira' 'Nairobi'] unique values

population: 
[1208333  866820 1453787  315943  143920  340671  841353  781263  867457
  459785  268002 1545714  393177  608599 1136187 1421932  987653  638289
  759164  610411 1056640 2417735  926976  621241  310327  990341 1163186
  454480  885711  666763  518560 2162202 1157873 1117840  901777  875689
 1867579  590

**Observations**
- All column values are standardized
- column names are all in order

In [128]:
#sanity check
population_df.columns

Index(['county_code', 'county', 'population', 'male', 'female', 'intersex',
       'sex_ratio', 'area_km2', 'density_per_km2', 'number_of_households',
       'avg_household_size', 'percent_of_national_pop'],
      dtype='object')

In [129]:
# save cleaned population data
population_df.to_csv('../cleaned_datasets/cleaned_population_density.csv',index=False)

## 3.6 Power Stations Data Cleaning

In [130]:
#Standardize column names
power_stations_df.columns = (
    power_stations_df.columns
    .str.strip()          
    .str.lower()          
    .str.replace(' ', '_')  
    .str.replace('-', '_')  
    .str.replace('/', '_')  
)
print("Standardized Column Names:\n", power_stations_df.columns.tolist())


Standardized Column Names:
 ['fid', 'objectid', 'type', 'name', 'capacity_m', 'coordinate', 'x_coord', 'y_coord', 'status', 'year', 'remarks', 'longitude', 'latitude']



This step ensures all column names are clean, consistent, and easy to use throughout the analysis

In [131]:
#check the data types
print(power_stations_df.dtypes)

fid             int64
objectid        int64
type           object
name           object
capacity_m    float64
coordinate     object
x_coord       float64
y_coord       float64
status         object
year           object
remarks        object
longitude     float64
latitude      float64
dtype: object


All columns are in their correct data types except we will have to change the year column to numeric fomart

In [132]:
#convert year to numeric
power_stations_df['year'] = pd.to_numeric(power_stations_df['year'], errors='coerce')
power_stations_df['year'].dtype

dtype('float64')

Observation

- The 'year' column has now been changed to a numeric data type.  
- Text-based columns (**type**, **name**, **status**, **remarks**, **coordinate**) are stored as object type.  
- The dataset is now correctly formatted for analysis.


In [133]:
power_stations_df.isnull().year.sum()

18

The 'year' column currently has 18 missing values.  
To make the dataset consistent, these missing values will be filled.  

In [134]:
power_stations_df['year'].fillna(power_stations_df['year'].median(), inplace=True)
power_stations_df.isnull().year.sum()


0

We will then drop the longitude, latitude, X_Coord and Y_Coord columns.

In [135]:
# Drop unecessary columns
power_stations_df = power_stations_df.drop(columns=['longitude', 'latitude', 'x_coord', 'y_coord', 'coordinate'], errors='ignore')

# Confirm remaining columns
power_stations_df.columns


Index(['fid', 'objectid', 'type', 'name', 'capacity_m', 'status', 'year',
       'remarks'],
      dtype='object')

Handle text columns

In [136]:
#Strip spaces and standardize text to lowercase
text_cols = ['name', 'type', 'status', 'remarks']

for col in text_cols:
    power_stations_df[col] = power_stations_df[col].astype(str).str.strip().str.lower()

#Review unique values in key categorical columns
print("Unique types:\n", power_stations_df['type'].unique())
print("\nUnique statuses:\n", power_stations_df['status'].unique())


Unique types:
 ['geothermal' 'hydroelectric' 'fossil fuels' 'wind' 'solar']

Unique statuses:
 ['operational' 'in development' 'planned' 'commissioned'
 'under construction']


The type and status columns are clean and consistent.

In [137]:
# save cleaned  power stations data
power_stations_df.to_csv('../cleaned_datasets/cleaned_power_stations.csv')

## 3.7 Athi River 

In [138]:
#creating a copy to retain the original dataset 
athi_data = athi_df.copy(deep=True)

#checking the columns
athi_data.columns

Index(['Time', 'Apparent power(+)(kVA)', 'Active power(+)(kW)',
       'Power factor(%)', 'A phase power factor(%)', 'B phase power factor(%)',
       'C phase power factor(%)', 'A phase current(A)', 'B phase current(A)',
       'C phase current(A)', 'A phase voltage(V)', 'B phase voltage(V)',
       'C phase voltage(V)'],
      dtype='object')

In [139]:
#standardizing the column names
col_map = {
    'Time': 'timestamp_local',
    'Apparent power(+)(kVA)': 'kva',
    'Active power(+)(kW)': 'kw',
    'Power factor(%)': 'pf',
    'A phase power factor(%)': 'pf_a',
    'B phase power factor(%)': 'pf_b',
    'C phase power factor(%)': 'pf_c',
    'A phase current(A)': 'ia',
    'B phase current(A)': 'ib',
    'C phase current(A)': 'ic',
    'A phase voltage(V)': 'va',
    'B phase voltage(V)': 'vb',
    'C phase voltage(V)': 'vc',
}

athi_data = athi_data.rename(columns=col_map)

#checking the columns
athi_data.columns

Index(['timestamp_local', 'kva', 'kw', 'pf', 'pf_a', 'pf_b', 'pf_c', 'ia',
       'ib', 'ic', 'va', 'vb', 'vc'],
      dtype='object')

In [140]:
# converting time from object to datetime
athi_data['timestamp_local'] = pd.to_datetime(athi_data['timestamp_local'], errors='coerce')

#confirming changes
athi_data.dtypes

timestamp_local    datetime64[ns]
kva                       float64
kw                        float64
pf                        float64
pf_a                      float64
pf_b                      float64
pf_c                      float64
ia                        float64
ib                        float64
ic                        float64
va                        float64
vb                        float64
vc                        float64
dtype: object

In [141]:
#checking for duplicates
print(f"The dataset has {athi_data.duplicated().sum()} duplicates.")

The dataset has 0 duplicates.


In [142]:
#checking for missing values
athi_data.isna().sum()

timestamp_local      0
kva                233
kw                 184
pf                  78
pf_a               156
pf_b               156
pf_c               156
ia                 156
ib                 156
ic                 156
va                 156
vb                 156
vc                 156
dtype: int64

In [143]:
# aggregating to hourly(use mean for power and max for currents and voltages to show stress peaks)
agg = {
    'kva':'mean', 'kw':'mean', 'pf':'mean', 'pf_a':'mean', 'pf_b':'mean', 'pf_c':'mean',
    'ia':'max', 'ib':'max', 'ic':'max',
    'va':'mean', 'vb':'mean', 'vc':'mean',
}

athi_data = athi_data.set_index('timestamp_local').sort_index().groupby(pd.Grouper(freq='H')).agg(agg)

athi_data= athi_data.reset_index()

print("Hourly shape:", athi_data.shape)

#previewing the first 5 rows
athi_data.head()

Hourly shape: (44436, 13)


,timestamp_local,kva,kw,pf,pf_a,pf_b,pf_c,ia,ib,ic,va,vb,vc
0,2020-10-01 01:00:00,87.28200,32.805,0.6375,0.7860,0.7960,0.8340,75.3,91.5,93.6,233.15,233.55,233.45
1,2020-10-01 02:00:00,14.91065,11.535,0.6365,0.7125,0.6070,0.7045,18.6,26.1,46.5,233.75,234.15,233.90
2,2020-10-01 03:00:00,34.59215,17.460,0.6750,0.8625,0.8115,0.8485,25.8,37.2,44.4,234.90,235.10,234.95
3,2020-10-01 04:00:00,266.87500,19.530,0.5225,0.7320,0.7045,0.7420,48.6,56.1,63.9,234.50,234.65,234.55
4,2020-10-01 05:00:00,12.97465,10.170,0.7290,0.5690,0.6655,0.6325,29.1,36.6,39.0,232.95,233.10,233.05


In [144]:
#handling power spike outliers (17,394kVA)

low = athi_data["kva"].quantile(0.01)
high = athi_data["kva"].quantile(0.99)

athi_data["kva"] = athi_data["kva"].clip(lower=low, upper=high)

In [145]:
# handling voltages to plausible range (200 to 260V)
for col in ['va','vb','vc']:
    athi_data[col] = athi_data[col].clip(lower=200, upper=260)
    
#checking the first 5 rows
athi_data.head()

,timestamp_local,kva,kw,pf,pf_a,pf_b,pf_c,ia,ib,ic,va,vb,vc
0,2020-10-01 01:00:00,87.282000,32.805,0.6375,0.7860,0.7960,0.8340,75.3,91.5,93.6,233.15,233.55,233.45
1,2020-10-01 02:00:00,14.910650,11.535,0.6365,0.7125,0.6070,0.7045,18.6,26.1,46.5,233.75,234.15,233.90
2,2020-10-01 03:00:00,34.592150,17.460,0.6750,0.8625,0.8115,0.8485,25.8,37.2,44.4,234.90,235.10,234.95
3,2020-10-01 04:00:00,135.843713,19.530,0.5225,0.7320,0.7045,0.7420,48.6,56.1,63.9,234.50,234.65,234.55
4,2020-10-01 05:00:00,12.974650,10.170,0.7290,0.5690,0.6655,0.6325,29.1,36.6,39.0,232.95,233.10,233.05


In [146]:
# 1) Resample to a full hourly grid (lowercase 'h' to avoid the warning)
agg = {
    'kva':'mean', 'kw':'mean', 'pf':'mean', 'pf_a':'mean', 'pf_b':'mean', 'pf_c':'mean',
    'ia':'max', 'ib':'max', 'ic':'max',            # keep stress peaks
    'va':'mean', 'vb':'mean', 'vc':'mean',
}

athi_hourly = (athi_data
               .set_index('timestamp_local')
               .sort_index()
               .resample('h')                      # full hourly index -> may introduce NaNs
               .agg(agg))

# 2) Gap diagnostics
key_cols = ['kw','kva','ia','ib','ic','va','vb','vc']
athi_hourly['row_missing_frac'] = athi_hourly[key_cols].isna().mean(axis=1)
athi_hourly['gap_flag_any'] = (athi_hourly['row_missing_frac'] > 0).astype(int)

# 3) Short-gap policy
# - interpolate small holes (<= 2 consecutive hours) for kw/kva (smooth, slowly varying)
# - forward/backward fill small holes (<= 2h) for voltages (nominal around 230V)
# - DO NOT fill long gaps -> keep NaN (we'll carry the gap flags forward)
for c in ['kw','kva']:
    athi_hourly[c] = athi_hourly[c].interpolate(method='time', limit=2)

for c in ['va','vb','vc']:
    athi_hourly[c] = athi_hourly[c].ffill(limit=2).bfill(limit=2)

# currents are "stress" signals; prefer not to fabricate them
# if you must, very small gaps can use max of neighbors:
for c in ['ia','ib','ic']:
    neigh_max = pd.concat([athi_hourly[c].shift(1), athi_hourly[c].shift(-1)], axis=1).max(axis=1)
    athi_hourly[c] = athi_hourly[c].fillna(neigh_max.where(neigh_max.notna()))  # fill only when neighbor exists

# 4) Recompute PF safely (handles PF NaNs created by resample)
def fix_pf(pf_series, kw, kva):
    s = pf_series.copy()
    s[(s < 0) | (s > 1)] = np.nan
    valid = (kw > 0.5) & (kva > 0.5) & s.isna()
    s.loc[valid] = (kw[valid] / kva[valid]).clip(0, 1)
    return s

athi_hourly['pf']   = fix_pf(athi_hourly['pf'],   athi_hourly['kw'], athi_hourly['kva'])
for ph in ['pf_a','pf_b','pf_c']:
    if ph in athi_hourly:
        athi_hourly[ph] = fix_pf(athi_hourly[ph], athi_hourly['kw'], athi_hourly['kva'])

# 5) Plant-off proxy (helps decide how to impute remaining NaNs)
plant_off = (
    (athi_hourly['kw'].fillna(0) < 0.1) &
    (athi_hourly[['ia','ib','ic']].fillna(0).max(axis=1) < 0.5)
)
athi_hourly['plant_off_flag'] = plant_off.astype(int)

# Optional: set kw/kva to 0 when plant is clearly off and values are missing
for c in ['kw','kva']:
    athi_hourly.loc[plant_off & athi_hourly[c].isna(), c] = 0.0

# 6) Voltage sanity (flag + cap AFTER imputation, as before)
for vcol in ['va','vb','vc']:
    if vcol in athi_hourly:
        flag = (athi_hourly[vcol] < 190) | (athi_hourly[vcol] > 260)
        athi_hourly[f'{vcol}_anomaly_flag'] = flag.astype(int)
        athi_hourly[vcol] = athi_hourly[vcol].clip(200, 260)

# 7) Guard your winsorization step (avoid turning a column into all-NaN if quantiles are NaN)
if athi_hourly['kva'].dropna().size > 50:
    low, high = athi_hourly['kva'].quantile([0.01, 0.99])
    if pd.notna(low) and pd.notna(high):
        athi_hourly['kva_outlier_flag'] = ((athi_hourly['kva'] < low) | (athi_hourly['kva'] > high)).astype(int)
        athi_hourly['kva'] = athi_hourly['kva'].clip(lower=low, upper=high)

# 8) Recreate imbalance & rolling features with data-availability requirements
eps = 1e-9
if {'ia','ib','ic'}.issubset(athi_hourly.columns):
    imean = athi_hourly[['ia','ib','ic']].mean(axis=1).replace(0, np.nan)
    athi_hourly['current_imbalance'] = (
        (athi_hourly[['ia','ib','ic']].max(axis=1) - athi_hourly[['ia','ib','ic']].min(axis=1)) / (imean + eps)
    )

if {'va','vb','vc'}.issubset(athi_hourly.columns):
    vmean = athi_hourly[['va','vb','vc']].mean(axis=1).replace(0, np.nan)
    athi_hourly['voltage_imbalance'] = (
        (athi_hourly[['va','vb','vc']].max(axis=1) - athi_hourly[['va','vb','vc']].min(axis=1)) / (vmean + eps)
    )

# rolling features with a minimum data fraction to avoid bias from NaNs
def roll(series, w, min_frac=0.7, func='mean'):
    minp = max(1, int(w * min_frac))
    if func == 'mean':
        return series.rolling(w, min_periods=minp).mean()
    if func == 'max':
        return series.rolling(w, min_periods=minp).max()

athi_hourly['kw_mean_6h']   = roll(athi_hourly['kw'], 6, 0.7, 'mean')
athi_hourly['kw_max_6h']    = roll(athi_hourly['kw'], 6, 0.7, 'max')
athi_hourly['kva_peak_24h'] = roll(athi_hourly['kva'],24, 0.7, 'max')

for c in ['va','vb','vc']:
    if c in athi_hourly:
        athi_hourly[f'{c}_sag_flag'] = (athi_hourly[c] < 205).astype(int)
sag_cols = [c for c in athi_hourly.columns if c.endswith('_sag_flag')]
athi_hourly['sag_events_24h'] = (
    athi_hourly[sag_cols].sum(axis=1).rolling(24, min_periods=12).sum()
)

# 9) Final tidy
athi_hourly = athi_hourly.reset_index().rename(columns={'index':'timestamp_local'})
athi_hourly['hour'] = athi_hourly['timestamp_local'].dt.hour
athi_hourly['dow']  = athi_hourly['timestamp_local'].dt.dayofweek
athi_hourly['is_weekend'] = (athi_hourly['dow'] >= 5).astype(int)

print("Remaining NaNs (top 10 columns):")
print(athi_hourly.isna().mean().sort_values(ascending=False).head(10))


Remaining NaNs (top 10 columns):
current_imbalance    0.473805
pf_a                 0.470069
pf_b                 0.470069
pf_c                 0.470069
pf                   0.470002
ia                   0.469462
ib                   0.469462
ic                   0.469462
vc                   0.393150
va                   0.393150
dtype: float64


In [147]:
athi_hourly.isna().sum()

timestamp_local          0
kva                   1709
kw                    1699
pf                   20885
pf_a                 20888
pf_b                 20888
pf_c                 20888
ia                   20861
ib                   20861
ic                   20861
va                   17470
vb                   17470
vc                   17470
row_missing_frac         0
gap_flag_any             0
plant_off_flag           0
va_anomaly_flag          0
vb_anomaly_flag          0
vc_anomaly_flag          0
kva_outlier_flag         0
current_imbalance    21054
voltage_imbalance    17470
kw_mean_6h              24
kw_max_6h               24
kva_peak_24h            37
va_sag_flag              0
vb_sag_flag              0
vc_sag_flag              0
sag_events_24h          11
hour                     0
dow                      0
is_weekend               0
dtype: int64

In [148]:
athi_hourly.head()

,timestamp_local,kva,kw,pf,pf_a,pf_b,pf_c,ia,ib,ic,...,kw_mean_6h,kw_max_6h,kva_peak_24h,va_sag_flag,vb_sag_flag,vc_sag_flag,sag_events_24h,hour,dow,is_weekend
0,2020-10-01 01:00:00,87.282000,32.805,0.6375,0.7860,0.7960,0.8340,75.3,91.5,93.6,...,NaN,NaN,NaN,0,0,0,NaN,1,3,0
1,2020-10-01 02:00:00,14.910650,11.535,0.6365,0.7125,0.6070,0.7045,18.6,26.1,46.5,...,NaN,NaN,NaN,0,0,0,NaN,2,3,0
2,2020-10-01 03:00:00,34.592150,17.460,0.6750,0.8625,0.8115,0.8485,25.8,37.2,44.4,...,NaN,NaN,NaN,0,0,0,NaN,3,3,0
3,2020-10-01 04:00:00,120.021454,19.530,0.5225,0.7320,0.7045,0.7420,48.6,56.1,63.9,...,20.3325,32.805,NaN,0,0,0,NaN,4,3,0
4,2020-10-01 05:00:00,12.974650,10.170,0.7290,0.5690,0.6655,0.6325,29.1,36.6,39.0,...,18.3000,32.805,NaN,0,0,0,NaN,5,3,0


In [149]:
#save cleaned athi river energy consumption data
athi_hourly.to_csv('../cleaned_datasets/cleaned_athi_energy_consumption.csv')

## 3.8 Garissa

In [150]:
#creating a copy to retain the original dataset 
garissa_data = garissa_df.copy(deep=True)

#checking the columns
garissa_data.columns

Index(['Time', 'Apparent power(+)(kVA)', 'Active power(+)(kW)',
       'Power factor(%)', 'A phase power factor(%)', 'B phase power factor(%)',
       'C phase power factor(%)', 'A phase current(A)', 'B phase current(A)',
       'C phase current(A)', 'A phase voltage(V)', 'B phase voltage(V)',
       'C phase voltage(V)'],
      dtype='object')

In [151]:
#standardizing the column names
col_map = {
    'Time': 'timestamp_local',
    'Apparent power(+)(kVA)': 'kva',
    'Active power(+)(kW)': 'kw',
    'Power factor(%)': 'pf',
    'A phase power factor(%)': 'pf_a',
    'B phase power factor(%)': 'pf_b',
    'C phase power factor(%)': 'pf_c',
    'A phase current(A)': 'ia',
    'B phase current(A)': 'ib',
    'C phase current(A)': 'ic',
    'A phase voltage(V)': 'va',
    'B phase voltage(V)': 'vb',
    'C phase voltage(V)': 'vc',
}

garissa_data = garissa_data.rename(columns=col_map)

#checking the columns
garissa_data.columns

Index(['timestamp_local', 'kva', 'kw', 'pf', 'pf_a', 'pf_b', 'pf_c', 'ia',
       'ib', 'ic', 'va', 'vb', 'vc'],
      dtype='object')

In [152]:
# converting time from object to datetime
garissa_data['timestamp_local'] = pd.to_datetime(garissa_data['timestamp_local'], errors='coerce')

#confirming changes
garissa_data.dtypes

timestamp_local    datetime64[ns]
kva                       float64
kw                        float64
pf                        float64
pf_a                      float64
pf_b                      float64
pf_c                      float64
ia                        float64
ib                        float64
ic                        float64
va                        float64
vb                        float64
vc                        float64
dtype: object

In [153]:
#checking for duplicates
print(f"The dataset has {garissa_data.duplicated().sum()} duplicates.")

The dataset has 0 duplicates.


In [154]:
#checking for missing values
garissa_data.isna().sum()

timestamp_local      0
kva                163
kw                 107
pf                  64
pf_a               102
pf_b               102
pf_c               102
ia                 102
ib                 102
ic                 102
va                 102
vb                 102
vc                 102
dtype: int64

In [155]:
# aggregating to hourly(use mean for power and max for currents and voltages to show stress peaks)
agg = {
    'kva':'mean', 'kw':'mean', 'pf':'mean', 'pf_a':'mean', 'pf_b':'mean', 'pf_c':'mean',
    'ia':'max', 'ib':'max', 'ic':'max',
    'va':'mean', 'vb':'mean', 'vc':'mean',
}

garissa_data = garissa_data.set_index('timestamp_local').sort_index().groupby(pd.Grouper(freq='H')).agg(agg)

garissa_data= garissa_data.reset_index()

print("Hourly shape:", garissa_data.shape)

#previewing the first 5 rows
garissa_data.head()

Hourly shape: (44436, 13)


,timestamp_local,kva,kw,pf,pf_a,pf_b,pf_c,ia,ib,ic,va,vb,vc
0,2020-10-01 01:00:00,25.27400,24.295,0.9615,0.9555,0.9540,0.9655,29.7,36.4,53.8,233.85,237.55,231.20
1,2020-10-01 02:00:00,34.43515,33.335,0.9650,0.9480,0.9575,0.9640,57.9,64.9,63.1,231.50,235.35,231.85
2,2020-10-01 03:00:00,27.63945,26.715,0.9640,0.9530,0.9510,0.9670,46.8,51.3,44.8,235.25,237.75,235.70
3,2020-10-01 04:00:00,29.25640,28.540,0.9725,0.9500,0.9570,0.9725,56.7,51.3,44.6,234.90,238.30,235.70
4,2020-10-01 05:00:00,33.91585,33.045,0.9725,0.9640,0.9590,0.9765,45.7,81.3,55.8,237.95,236.95,234.80


In [156]:
#handling power spike outliers (17,394kVA)

low = garissa_data["kva"].quantile(0.01)
high = garissa_data["kva"].quantile(0.99)

garissa_data["kva"] = garissa_data["kva"].clip(lower=low, upper=high)

In [157]:
# handling voltages to plausible range (200 to 260V)
for col in ['va','vb','vc']:
    garissa_data[col] = garissa_data[col].clip(lower=200, upper=260)
    
#checking the first 5 rows
garissa_data.head()

,timestamp_local,kva,kw,pf,pf_a,pf_b,pf_c,ia,ib,ic,va,vb,vc
0,2020-10-01 01:00:00,25.27400,24.295,0.9615,0.9555,0.9540,0.9655,29.7,36.4,53.8,233.85,237.55,231.20
1,2020-10-01 02:00:00,34.43515,33.335,0.9650,0.9480,0.9575,0.9640,57.9,64.9,63.1,231.50,235.35,231.85
2,2020-10-01 03:00:00,27.63945,26.715,0.9640,0.9530,0.9510,0.9670,46.8,51.3,44.8,235.25,237.75,235.70
3,2020-10-01 04:00:00,29.25640,28.540,0.9725,0.9500,0.9570,0.9725,56.7,51.3,44.6,234.90,238.30,235.70
4,2020-10-01 05:00:00,33.91585,33.045,0.9725,0.9640,0.9590,0.9765,45.7,81.3,55.8,237.95,236.95,234.80


In [158]:
#checking shape
garissa_data.describe().T

,count,mean,min,25%,50%,75%,max,std
timestamp_local,44436,2023-04-14 18:30:00,2020-10-01 01:00:00,2022-01-06 21:45:00,2023-04-14 18:30:00,2024-07-20 15:15:00,2025-10-26 12:00:00,NaN
kva,21563.0,30.448663,0.0,22.639775,30.2389,37.948675,64.191926,12.117002
kw,21574.0,29.10591,0.0,21.26125,29.115,36.69875,84.68,11.958107
pf,21595.0,0.953989,0.019,0.9635,0.978,0.989,1.0,0.085533
pf_a,21593.0,0.92523,0.0,0.9205,0.952,0.97,1.0,0.081795
pf_b,21593.0,0.909453,0.0,0.908,0.942,0.9595,1.0,0.091512
pf_c,21593.0,0.958155,0.0,0.9625,0.974,0.9835,1.0,0.060085
ia,21593.0,51.014861,0.0,32.3,47.3,66.2,182.1,26.484802
ib,21593.0,56.866322,0.0,38.7,55.4,72.8,192.2,26.247303
ic,21593.0,57.437285,0.0,43.3,56.0,70.4,174.9,21.766843


In [159]:
#save cleaned garissa energy consumption data
garissa_data.to_csv('../cleaned_datasets/cleaned_garissa_energy_consumption.csv')

## 3.9 Energy Consumption Data Cleaning

In [160]:
#creating a copy to retain the original dataset 
energy_df = consumption_df.copy(deep=True)

#checking the columns
energy_df.columns

Index(['RoomArea', 'NumberofAppliances', 'Outside Temperature',
       'InsulationThickness', 'BuildingType', 'HVACSystem',
       'AverageTemperature', 'EnergyConsumption'],
      dtype='object')

In [161]:
#lowercasing the column names

energy_df.columns = energy_df.columns.str.lower().str.replace(" ", "")

energy_df.columns

Index(['roomarea', 'numberofappliances', 'outsidetemperature',
       'insulationthickness', 'buildingtype', 'hvacsystem',
       'averagetemperature', 'energyconsumption'],
      dtype='object')

In [162]:
#checking for duplicates
print(f"The dataset has {energy_df.duplicated().sum()} duplicates.")

The dataset has 0 duplicates.


In [163]:
#checking for missing values
energy_df.isna().sum()

roomarea               0
numberofappliances     0
outsidetemperature     0
insulationthickness    0
buildingtype           0
hvacsystem             0
averagetemperature     0
energyconsumption      0
dtype: int64

In [164]:
#cap indoor temperatures to plausible values(10 to 32°C)

energy_df["averagetemperature"] = energy_df["averagetemperature"].clip(lower=10, upper=32)

In [165]:
#remove outliers in energy consumption by limiting to 1st and 99th percentile
low = energy_df["energyconsumption"].quantile(0.01)
high = energy_df["energyconsumption"].quantile(0.99)

energy_df["energyconsumption"] = energy_df["energyconsumption"].clip(lower=low, upper=high)

In [166]:
#one-hot-encode categoricals
energy_df = pd.get_dummies(energy_df, columns=['buildingtype','hvacsystem'], drop_first=True)

#preview the first 5 rows
energy_df.head()

,roomarea,numberofappliances,outsidetemperature,insulationthickness,averagetemperature,energyconsumption,buildingtype_Residential,hvacsystem_Split AC,hvacsystem_Window AC
0,302,5,-9.638784,7.227564,10.000000,1828.820920,True,False,True
1,470,17,13.536505,1.086168,15.267517,2307.834235,True,False,True
2,306,17,13.948559,3.590366,12.029111,1591.374161,False,False,True
3,271,18,5.866391,1.191942,10.000000,1463.371403,True,True,False
4,388,12,-0.518886,3.750820,10.000000,2198.909856,True,False,False


In [167]:
#save cleaned energy consumption data
energy_df.to_csv('../cleaned_datasets/cleaned_energy_consumption.csv')

## 3.10 Primary Substations Data Cleaning

Standardising column names.

We will then rename the columns for easier analysis and consistency throughout our analysis by:
   
   1. Converting names to lowercases.
   2. Remove spaces and hyphens with underscores.
   3. Remove unecessary numbers and characters.
  

In [168]:
#Standardize column names
substations_df.columns = (
    substations_df.columns
    .str.strip()          
    .str.lower()          
    .str.replace(' ', '_')  
    .str.replace('-', '_')  
    .str.replace('/', '_')  
)
print("Standardized Column Names:\n", substations_df.columns.tolist())


Standardized Column Names:
 ['fid', 'objectid', 'rcc1', 'county2', 'branch3', 'name8', 'name_ims11', 'ownershi24', 'availabi39', 'street40', 'location41', 'outgoing48', 'manned50', 'availabi51', 'earthing56', 'remote_t57', 'longitude', 'latitude']


In [169]:
#Check for Outliers in Numeric Columns

#Select only numeric columns
numeric_cols = substations_df.select_dtypes(include=['float64', 'int64']).columns

#Display summary statistics for numeric columns
print(substations_df[numeric_cols].describe())


              fid    objectid      longitude      latitude
count  264.000000  264.000000     264.000000  2.640000e+02
mean   131.500000  132.500000  265009.938581  9.894892e+06
std     76.354437   76.354437  203588.346468  1.796403e+05
min      0.000000    1.000000 -658779.677285  9.491473e+06
25%     65.750000   66.750000  114504.018069  9.850965e+06
50%    131.500000  132.500000  262007.938453  9.901779e+06
75%    197.250000  198.250000  335493.072483  9.994135e+06
max    263.000000  264.000000  817297.836855  1.046139e+07


No statistical outliers in the data structure, but longitude and latitude are not in geographic degrees.

In this step, we confirm whether the coordinate values for longitude and latitude are in the correct format.
Since the  summary shows large values (in the hundreds of thousands and millions), they likely represent projected coordinates (e.g., UTM) instead of standard decimal degrees.

In [170]:
#Display min and max values for longitude and latitude
print("Longitude range:", substations_df['longitude'].min(), "to", substations_df['longitude'].max())
print("Latitude range:", substations_df['latitude'].min(), "to", substations_df['latitude'].max())

Longitude range: -658779.677285241 to 817297.8368552485
Latitude range: 9491473.345593208 to 10461394.799870895


These numbers are too large for normal geographic coordinates (longitude usually ranges from -180 to +180; latitude from -90 to +90).
So the coordinates are projected coordinates, probably in UTM (Universal Transverse Mercator) format measured in meters, not degrees.

In [171]:
#Convert projected coordinates (UTM) to latitude and longitude in degrees

#Create a GeoDataFrame using UTM coordinates
substations_df = gpd.GeoDataFrame(
    substations_df,
    geometry=gpd.points_from_xy(substations_df["longitude"], substations_df["latitude"]),
    crs="EPSG:21037"  
)

#Convert (reproject) to WGS84 coordinate system (lat/lon in degrees)
substations_df = substations_df.to_crs(epsg=4326)

#Extract the converted coordinates into new columns
substations_df["longitude_deg"] = substations_df.geometry.x
substations_df["latitude_deg"] = substations_df.geometry.y

#Confirm the result
print(substations_df[["longitude", "latitude", "longitude_deg", "latitude_deg"]].head())


       longitude      latitude  longitude_deg  latitude_deg
0  257504.944880  9.859384e+06      36.821623     -1.273974
1  258513.019750  9.857610e+06      36.830665     -1.290018
2  257096.063665  9.857859e+06      36.817939     -1.287761
3  256961.749868  9.858320e+06      36.816736     -1.283586
4  258743.239612  9.856004e+06      36.832720     -1.304544


The location coordinates are now in degrees fomart

In [172]:
# Replace old columns with corrected coordinates
substations_df['longitude'] = substations_df['longitude_deg']
substations_df['latitude'] = substations_df['latitude_deg']

# Drop temporary columns if not needed
substations_df.drop(columns=['longitude_deg', 'latitude_deg', 'geometry'], inplace=True, errors='ignore')

# Confirm final structure
substations_df[['longitude', 'latitude']].head()


,longitude,latitude
0,36.821623,-1.273974
1,36.830665,-1.290018
2,36.817939,-1.287761
3,36.816736,-1.283586
4,36.832720,-1.304544


Data validation.

In [173]:
#Check for unique IDs
print("fid unique:", substations_df["fid"].is_unique)
print("objectid unique:", substations_df["objectid"].is_unique)

#Check valid coordinate ranges (Kenya range)
print("Longitude out of range:")
print(substations_df[(substations_df["longitude"] < 33) | (substations_df["longitude"] > 42)])

print("\nLatitude out of range:")
print(substations_df[(substations_df["latitude"] < -5) | (substations_df["latitude"] > 5)])


fid unique: True
objectid unique: True
Longitude out of range:
     fid  objectid rcc1 county2 branch3                    name8  \
255  255       256                                2ND 7.5 MVA TX   
256  256       257                       MACHAKOS  2ND 7.5MVA TX   

                       name_ims11 ownershi24 availabi39 street40 location41  \
255  2ND 7.5 MVA TX ( 33 / 11 KV)       KPLC                       MACHAKOS   
256  MACHAKOS  2ND 7.5MVA TX ( 33       KPLC                       MACHAKOS   

    outgoing48 manned50 availabi51 earthing56 remote_t57  longitude  latitude  
255         11       NO                   YES             28.702063 -0.041528  
256         11       NO                   YES             28.644412 -0.050418  

Latitude out of range:
Empty DataFrame
Columns: [fid, objectid, rcc1, county2, branch3, name8, name_ims11, ownershi24, availabi39, street40, location41, outgoing48, manned50, availabi51, earthing56, remote_t57, longitude, latitude]
Index: []


Two rows (FID 255 and 256) have longitude_deg ≈ 28.7° and 28.6°,
which are outside Kenya’s normal longitude range (33° to 42°).

In [174]:
#Identify the invalid rows
invalid_coords = substations_df[(substations_df["longitude"] < 33) | (substations_df["longitude"] > 42)]
print(invalid_coords[["fid", "county2", "longitude", "latitude"]])

#Fix the invalid longitude values
substations_df.loc[substations_df["fid"].isin([255, 256]), "longitude"] = 37.25
substations_df.loc[substations_df["fid"].isin([255, 256]), "latitude"] = -1.5

#Verify the fix
print("\nAfter fixing:")
print(substations_df.loc[substations_df["fid"].isin([255, 256]), ["fid", "county2", "longitude", "latitude"]])

#Confirm there are no more out-of-range values
print("\nRemaining longitude out of range:")
print(substations_df[(substations_df["longitude"] < 33) | (substations_df["longitude"] > 42)])


     fid county2  longitude  latitude
255  255          28.702063 -0.041528
256  256          28.644412 -0.050418

After fixing:
     fid county2  longitude  latitude
255  255              37.25      -1.5
256  256              37.25      -1.5

Remaining longitude out of range:
Empty DataFrame
Columns: [fid, objectid, rcc1, county2, branch3, name8, name_ims11, ownershi24, availabi39, street40, location41, outgoing48, manned50, availabi51, earthing56, remote_t57, longitude, latitude]
Index: []


The longitude and latitude values for records with FID 255 and 256 were successfully corrected to valid geographic coordinates within Kenya’s range. No longitude or latitude values are now out of range, confirming that all coordinate data is valid and consistent.

In [175]:
substations_df.columns

Index(['fid', 'objectid', 'rcc1', 'county2', 'branch3', 'name8', 'name_ims11',
       'ownershi24', 'availabi39', 'street40', 'location41', 'outgoing48',
       'manned50', 'availabi51', 'earthing56', 'remote_t57', 'longitude',
       'latitude'],
      dtype='object')

In [176]:
# save cleaned primary substations data
substations_df.to_csv('../cleaned_datasets/cleaned_primary_substations.csv')

## 3.11 Transformers Data Cleaning

In [177]:
# Check nulls
transformers_df.isna().sum()

FID            0
OBJECTID       0
RCC1           0
County2        0
Branch3        0
Substati9      0
DCS_Cate15     0
Modifyin25     0
Origin_o27     0
Feeder_o28     0
Substati29     0
HT_Isola32     0
Road_Str38    23
Physical39    14
Road           3
longitude      0
latitude       0
dtype: int64

In [178]:
# Check duplicates
transformers_df.duplicated().sum()

0

In [179]:
# Fill missing values with "Unknown"
transformers_df = transformers_df.fillna("Unknown")

# Check
print(transformers_df.isna().sum())

FID           0
OBJECTID      0
RCC1          0
County2       0
Branch3       0
Substati9     0
DCS_Cate15    0
Modifyin25    0
Origin_o27    0
Feeder_o28    0
Substati29    0
HT_Isola32    0
Road_Str38    0
Physical39    0
Road          0
longitude     0
latitude      0
dtype: int64


In [180]:
# save cleaned distribution transformers data
transformers_df.to_csv('../cleaned_datasets/cleaned_distribution_transformers.csv')

## 3.12 Powerplants Data cleaning

In [181]:
 #Column name normalization + trim strings
powerplants_df.columns = powerplants_df.columns.str.strip().str.lower().str.replace(" ", "_")

# Convert object columns: strip whitespace and convert textual nulls to np.nan
for c in powerplants_df.select_dtypes(include="object").columns:
    powerplants_df[c] = powerplants_df[c].astype(str).str.strip()
    powerplants_df[c] = powerplants_df[c].replace(
        {'': np.nan, 'nan': np.nan, 'none': np.nan, 'null': np.nan, 'n/a': np.nan, 'na': np.nan}
    )

print("Columns after normalization:", powerplants_df.columns.tolist())


Columns after normalization: ['plant', 'status', 'sum_mw', 'country', 'lat', 'lon', 'gen_type', 'iso']


In [182]:
#Fix numeric dtypes and report parsing problems
numeric_candidates = ["sum_mw", "lat", "lon"]
for col in numeric_candidates:
    # remove commas, stray characters, convert to numeric
    powerplants_df[col] = powerplants_df[col].astype(str).str.replace(",","").str.strip()
    powerplants_df[col] = pd.to_numeric(powerplants_df[col], errors="coerce")

# Report
print("Numeric conversion status:")
display(powerplants_df[numeric_candidates].dtypes)
print("Non-null counts for numeric fields:")
display(powerplants_df[numeric_candidates].notnull().sum())


Numeric conversion status:


sum_mw    float64
lat       float64
lon       float64
dtype: object

Non-null counts for numeric fields:


sum_mw    32
lat       32
lon       32
dtype: int64

In [183]:

# Clean categorical labels
for c in ["status", "gen_type", "country", "iso", "plant"]:
    powerplants_df[c] = powerplants_df[c].astype(str).str.upper().str.strip()


In [184]:
#sanity check
powerplants_df.columns

Index(['plant', 'status', 'sum_mw', 'country', 'lat', 'lon', 'gen_type',
       'iso'],
      dtype='object')

In [185]:
#save cleaned powerplants data
powerplants_df.to_csv('../cleaned_datasets/cleaned_powerplants.csv')

## 3.13 Transmission Network Data Cleaning

In [186]:
#checking null values
network_df.isnull().sum()

COUNTRY        0
CNTRY_NAME     0
VOLTAGE_KV     0
FROM_NM        0
TO_NM          0
STATUS         0
SOURCES        1
PROJECT_NM    45
dtype: int64

In [187]:
#check for duplicates
network_df.duplicated().sum()

0

In [188]:
#fill missing values in SOURCES and PROJECT_NM columns with 'Unknown'
network_df['SOURCES'] = network_df['SOURCES'].fillna('Unknown')
network_df['PROJECT_NM'] = network_df['PROJECT_NM'].fillna('Unknown')


In [189]:
#checking null values
network_df.isnull().sum()

COUNTRY       0
CNTRY_NAME    0
VOLTAGE_KV    0
FROM_NM       0
TO_NM         0
STATUS        0
SOURCES       0
PROJECT_NM    0
dtype: int64

In [190]:
for col in network_df.select_dtypes(include='object').columns:
    network_df[col] = network_df[col].str.strip().str.title()


In [191]:
#save cleaned transmission network data
network_df.to_csv('../cleaned_datasets/cleaned_transmission_network.csv')

## 3.14 Data Cleaning - Additional Weather

In [192]:
#checking for null values
new_weather_df.isna().sum()

COUNTY                         0
time                           0
cape_mean_6h_mean              0
cape_mean_12h_mean             0
cape_mean_24h_mean             0
wind_mean_6h_mean              0
wind_mean_12h_mean             0
wind_mean_24h_mean             0
wind_max_6h_max                0
wind_max_12h_max               0
wind_max_24h_max               0
cloud_mean_6h_mean             0
cloud_mean_12h_mean            0
cloud_mean_24h_mean            0
total_precipitation_mm_mean    0
total_precipitation_mm_max     0
total_precipitation_mm_sum     0
hour_first                     0
day_of_week_first              0
month_first                    0
is_weekend_first               0
is_holiday_first               0
season_first                   0
dtype: int64

In [193]:
#checking for duplicates
new_weather_df.duplicated().sum()

0

In [194]:
#lowercasing the column names

new_weather_df.columns = new_weather_df.columns.str.lower().str.replace(" ", "_")

new_weather_df.columns

Index(['county', 'time', 'cape_mean_6h_mean', 'cape_mean_12h_mean',
       'cape_mean_24h_mean', 'wind_mean_6h_mean', 'wind_mean_12h_mean',
       'wind_mean_24h_mean', 'wind_max_6h_max', 'wind_max_12h_max',
       'wind_max_24h_max', 'cloud_mean_6h_mean', 'cloud_mean_12h_mean',
       'cloud_mean_24h_mean', 'total_precipitation_mm_mean',
       'total_precipitation_mm_max', 'total_precipitation_mm_sum',
       'hour_first', 'day_of_week_first', 'month_first', 'is_weekend_first',
       'is_holiday_first', 'season_first'],
      dtype='object')

In [195]:
#changing time to date time
new_weather_df['time'] = pd.to_datetime(new_weather_df['time'], errors='coerce')

# Create separate date and time columns
new_weather_df['date'] = new_weather_df['time'].dt.date
new_weather_df['time'] = new_weather_df['time'].dt.time

#confirming changes
new_weather_df.dtypes

county                          object
time                            object
cape_mean_6h_mean              float64
cape_mean_12h_mean             float64
cape_mean_24h_mean             float64
wind_mean_6h_mean              float64
wind_mean_12h_mean             float64
wind_mean_24h_mean             float64
wind_max_6h_max                float64
wind_max_12h_max               float64
wind_max_24h_max               float64
cloud_mean_6h_mean             float64
cloud_mean_12h_mean            float64
cloud_mean_24h_mean            float64
total_precipitation_mm_mean    float64
total_precipitation_mm_max     float64
total_precipitation_mm_sum     float64
hour_first                       int64
day_of_week_first                int64
month_first                      int64
is_weekend_first                 int64
is_holiday_first                 int64
season_first                    object
date                            object
dtype: object

In [196]:
# save cleaned new weather data
new_weather_df.to_csv('../cleaned_datasets/new_weather_data.csv',index=False)

### Merging the datasets

In [ ]:
# def clean_county(df):
#     if 'county' in df.columns:
#         df['county'] = (
#             df['county']
#             .astype(str)
#             .str.strip()
#             .str.lower()
#             .str.replace(r'[^a-z0-9 ]', '', regex=True)
#             .str.replace(r'\s+', ' ', regex=True)
#         )
#     return df

# # Apply to all
# outages_df = clean_county(hourly_summary)
# population_df = clean_county(population_df)
# vegetation_df = clean_county(vegetation_df)
# weather_df = clean_county(weather_df)
# weather_hourly_df = clean_county(weather_hourly_df)
# new_weather_df = clean_county(new_weather_df)


In [204]:
#sanity check - confirm all counties match
print("Population counties:\n", population_df['county'].unique()[:47])
print("Vegetation counties:\n", vegetation_df['county'].unique()[:47])
print("Weather counties:\n", weather_df['county'].unique()[:47])
print("outages counties:\n", outages_df['county'].unique()[:47])
print("new weather counties:\n", new_weather_df['county'].unique()[:47])
print("weather Hourly  counties:\n", weather_hourly_df['county'].unique()[:47])

Population counties:
 ['mombasa' 'kwale' 'kilifi' 'tana river' 'lamu' 'taita taveta' 'garissa'
 'wajir' 'mandera' 'marsabit' 'isiolo' 'meru' 'tharaka nithi' 'embu'
 'kitui' 'machakos' 'makueni' 'nyandarua' 'nyeri' 'kirinyaga' 'muranga'
 'kiambu' 'turkana' 'west pokot' 'samburu' 'trans nzoia' 'uasin gishu'
 'elgeyo marakwet' 'nandi' 'baringo' 'laikipia' 'nakuru' 'narok' 'kajiado'
 'kericho' 'bomet' 'kakamega' 'vihiga' 'bungoma' 'busia' 'siaya' 'kisumu'
 'homa bay' 'migori' 'kisii' 'nyamira' 'nairobi']
Vegetation counties:
 ['baringo' 'bomet' 'bungoma' 'busia' 'embu' 'garissa' 'homa bay' 'isiolo'
 'kajiado' 'kakamega' 'keiyomarakwet' 'kericho' 'kiambu' 'kilifi'
 'kirinyaga' 'kisii' 'kisumu' 'kitui' 'kwale' 'laikipia' 'lamu' 'machakos'
 'makueni' 'mandera' 'marsabit' 'meru' 'migori' 'mombasa' 'muranga'
 'nairobi' 'nakuru' 'nandi' 'narok' 'nyamira' 'nyandarua' 'nyeri'
 'samburu' 'siaya' 'taita taveta' 'tana river' 'tharaka' 'trans nzoia'
 'turkana' 'uasin gishu' 'vihiga' 'wajir' 'west poko

**Observations**
- we will standardize the county names so they are similar
    - This makes it easier to merge by county
    

In [ ]:


# County Normalization 
county_map = {
    "homabay": "homa bay",
    "keiyomarakwet": "elgeyo marakwet",
    "elgeyomarakwet": "elgeyo marakwet",
    "tharakanithi": "tharaka nithi",
    "tharaka": "tharaka nithi",
    "taitataveta": "taita taveta",
    "nkubu": "meru", 
}

def normalize_counties(df, col_name='county'):
    if col_name in df.columns:
        df[col_name] = (
            df[col_name]
            .astype(str).str.lower().str.strip()
            .str.replace(r'[^a-z0-9 ]','', regex=True)
            .str.replace(r'\s+', ' ', regex=True) 
            .replace(county_map)
        )
    return df

# Load and normalize all datasets
outages_df = normalize_counties(pd.read_csv('../cleaned_datasets/cleaned_outages.csv'), 'county')
population_df = normalize_counties(pd.read_csv('../cleaned_datasets/cleaned_population_density.csv'), 'county')
vegetation_df = normalize_counties(pd.read_csv('../cleaned_datasets/cleaned_vegetation_data.csv'), 'county')
new_weather_df = normalize_counties(pd.read_csv('../cleaned_datasets/new_weather_data.csv'), 'county')

# fix county column names in infrastructure datasets
transformers_df = normalize_counties(
    pd.read_csv('../cleaned_datasets/cleaned_distribution_transformers.csv').rename(columns={'County2': 'county'}), 
    'county'
)
substations_df = normalize_counties(
    pd.read_csv('../cleaned_datasets/cleaned_primary_substations.csv').rename(columns={'county2': 'county'}), 
    'county'
)

# create base table from new weather data
# recreate hour timestamp
new_weather_df['hour'] = pd.to_datetime(
    new_weather_df['date'].astype(str) + ' ' + new_weather_df['time'].astype(str),
    errors='coerce'
)
#drop rows with errors in hour
master_df = new_weather_df.dropna(subset=['hour']).drop(columns=['date', 'time'], errors='ignore')
#rename hour to date for next merge
master_df['date'] = master_df['hour'].dt.date 


# Creating and Joining Outage Labels
outages_df['start_datetime'] = pd.to_datetime(outages_df['date'].astype(str) + ' ' + outages_df['start_time'].astype(str), errors='coerce')
outages_df['end_datetime'] = pd.to_datetime(outages_df['date'].astype(str) + ' ' + outages_df['end_time'].astype(str), errors='coerce')

# Handle overnight outages
overnight_mask = outages_df['end_datetime'] < outages_df['start_datetime']
outages_df.loc[overnight_mask, 'end_datetime'] += pd.Timedelta(days=1)
outages_df = outages_df.dropna(subset=['start_datetime', 'end_datetime']) 

# Expand events to hourly rows
hourly_outages_list = []
for _, row in outages_df.iterrows():
    hours = pd.date_range(start=row['start_datetime'].floor('H'), end=row['end_datetime'].ceil('H'), freq='h')
    hourly_outages_list.append(pd.DataFrame({'county': row['county'], 'hour': hours, 'planned_outage': 1}))

if hourly_outages_list:
    hourly_outage_labels = pd.concat(hourly_outages_list).drop_duplicates(subset=['county', 'hour'])
    # Join labels to master
    master_df = master_df.merge(hourly_outage_labels, on=['county', 'hour'], how='left')
    master_df['planned_outage'] = master_df['planned_outage'].fillna(0).astype(int)
else:
    master_df['planned_outage'] = 0

# Joining population, vegetation, and infrastructure
vegetation_df['date'] = pd.to_datetime(vegetation_df['date'], errors='coerce').dt.date

# Create infrastructure count mappings 
transformer_counts = transformers_df.groupby('county').size()
substation_counts = substations_df.groupby('county').size()

# Merge population data
master_df = master_df.merge(population_df, on='county', how='left')

# Merge vegetation 
master_df = master_df.merge(vegetation_df, on=['county', 'date'], how='left', suffixes=('_weather', '_veg'))

# Sort by county and time 
master_df = master_df.sort_values(by=['county', 'hour'])

# Forward-fill sparse vegetation data
ndvi_cols = ['ndvi_mean', 'ndvi_std', 'ndvi_min', 'ndvi_max']
master_df[ndvi_cols] = master_df.groupby('county')[ndvi_cols].ffill()

# Map infrastructure counts
master_df['transformer_count_per_county'] = master_df['county'].map(transformer_counts).fillna(0)
master_df['substation_count_per_county'] = master_df['county'].map(substation_counts).fillna(0)

# sanity check
print("\nFinal dataset shape: {}\n".format(master_df.shape))
print("\nFinal Dataset Info:\n")
master_df.info()
print("\nFinal Dataset Head:\n")
display(master_df.head())

# Save the final matrix
master_df.to_csv('../cleaned_datasets/merged_df.csv', index=False)



Final dataset shape: (726110, 43)


Final Dataset Info:

<class 'pandas.core.frame.DataFrame'>
Index: 726110 entries, 0 to 726109
Data columns (total 43 columns):
 #   Column                        Non-Null Count   Dtype         
---  ------                        --------------   -----         
 0   county                        726110 non-null  object        
 1   cape_mean_6h_mean             726110 non-null  float64       
 2   cape_mean_12h_mean            726110 non-null  float64       
 3   cape_mean_24h_mean            726110 non-null  float64       
 4   wind_mean_6h_mean             726110 non-null  float64       
 5   wind_mean_12h_mean            726110 non-null  float64       
 6   wind_mean_24h_mean            726110 non-null  float64       
 7   wind_max_6h_max               726110 non-null  float64       
 8   wind_max_12h_max              726110 non-null  float64       
 9   wind_max_24h_max              726110 non-null  float64       
 10  cloud_mean_6h_mean         

,county,cape_mean_6h_mean,cape_mean_12h_mean,cape_mean_24h_mean,wind_mean_6h_mean,wind_mean_12h_mean,wind_mean_24h_mean,wind_max_6h_max,wind_max_12h_max,wind_max_24h_max,...,number_of_households,avg_household_size,percent_of_national_pop,Unnamed: 0,ndvi_mean,ndvi_std,ndvi_min,ndvi_max,transformer_count_per_county,substation_count_per_county
0,baringo,149.416667,149.416667,149.416667,0.809555,0.809555,0.809555,1.235864,1.235864,1.235864,...,142518,4.7,1.40,0.0,0.482464,0.155480,-0.1993,0.9450,1028.0,3.0
1,baringo,154.679167,154.679167,154.679167,0.825501,0.825501,0.825501,1.235864,1.235864,1.235864,...,142518,4.7,1.40,0.0,0.482464,0.155480,-0.1993,0.9450,1028.0,3.0
2,baringo,152.552778,152.552778,152.552778,0.844295,0.844295,0.844295,1.324105,1.324105,1.324105,...,142518,4.7,1.40,0.0,0.482464,0.155480,-0.1993,0.9450,1028.0,3.0
3,baringo,146.929167,146.929167,146.929167,0.844887,0.844887,0.844887,1.394833,1.394833,1.394833,...,142518,4.7,1.40,0.0,0.482464,0.155480,-0.1993,0.9450,1028.0,3.0
4,baringo,139.790000,139.790000,139.790000,0.844379,0.844379,0.844379,1.414635,1.414635,1.414635,...,142518,4.7,1.40,0.0,0.482464,0.155480,-0.1993,0.9450,1028.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
726105,west pokot,198.518092,153.069627,243.722588,2.169606,1.766332,1.512562,3.716106,3.716106,4.044290,...,116182,5.3,1.31,NaN,0.580714,0.140128,-0.1816,0.9122,385.0,2.0
726106,west pokot,208.469846,152.179002,239.133909,1.974275,1.808966,1.484187,3.634701,3.716106,3.716106,...,116182,5.3,1.31,NaN,0.580714,0.140128,-0.1816,0.9122,385.0,2.0
726107,west pokot,220.718202,158.334978,232.945175,1.779532,1.847872,1.471427,3.391531,3.716106,3.716106,...,116182,5.3,1.31,NaN,0.580714,0.140128,-0.1816,0.9122,385.0,2.0
726108,west pokot,228.638706,175.694627,224.573328,1.650664,1.883596,1.472720,3.790390,3.790390,3.790390,...,116182,5.3,1.31,NaN,0.580714,0.140128,-0.1816,0.9122,385.0,2.0
